## gseapy analysis of CM7_CM1E2d56col_unenr123_rawextract_2017
gsea algorithm: http://software.broadinstitute.org/gsea/doc/GSEAUserGuideFrame.html  
gseapy: https://media.readthedocs.org/pdf/gseapy/latest/gseapy.pdf  
https://github.com/stuppie/CM7_CM1E2d56col_unenr123_rawextract_2017/blob/master/scripts/gseapy.ipynb

In [1]:
import sys
import os
from itertools import chain
from collections import defaultdict
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('precision', 3)
pd.set_option('display.max_colwidth', -1)
import gseapy as gp
import goatools

obodag = goatools.obo_parser.GODag('go-basic.obo')

load obo file go-basic.obo
go-basic.obo: fmt(1.2) rel(2017-03-16) 48,478 GO Terms


In [2]:
sys.path.insert(0, "/home/gstupp/projects/metaproteomics")
from metaproteomics import utils
#from metaproteomics.analysis import build_loci

BASE = '../out/'
grouped_loci = utils.load(os.path.join(BASE,"grouped_loci_filt_norm.pkl.gz"))

In [3]:
def make_go2Gene_map(grouped_loci, ontology='MF'):    

    ontology_map = {'MF': 'molecular_function', 'BP': 'biological_process', 'CC': 'cellular_component'}
    
    out = defaultdict(set)    
    for l in grouped_loci:
        if 'go' in l.annotations:
            for go in l.annotations['go']:
                if obodag[go].namespace == ontology_map[ontology]:
                    out[go].add(l.cluster_id)
                    for parent in obodag[go].get_all_parents():
                        if obodag[parent].namespace == ontology_map[ontology]:
                            out[parent].add(l.cluster_id)
                
    return dict(out)

def filter_go2gene_map(go_locus):
    
    # Remove "very broad" gene sets. Arbitrary definition: gene sets that emcompass >50% of all IDs
    all_ids = set(chain(*go_locus.values()))
    go_locus = {key: value for (key, value) in go_locus.items() if len(value) / len(all_ids) <= 0.5}

    # Remove terms with less than 5 members: changed from 10 to 5 becasue small #s of proteins compared
    # to what you would find wiht genes
    go_locus = {key: value for (key, value) in go_locus.items() if len(value) >= 5}

    # Remove child terms with identical gene sets as their parents
    to_remove = set()
    for parent in go_locus.keys():
        # If child term has exact same members as parent, remove
        child_ids = [x.id for x in obodag[parent].children if x.id in go_locus.keys()]
        for child in child_ids:
            if go_locus[child] == go_locus[parent]:
                to_remove.add(child)
    go_locus = {key: value for (key, value) in go_locus.items() if key not in to_remove}

    # Remove sibling terms with identical gene sets
    to_remove = set()
    to_keep = set()
    for brother in go_locus.keys():
        to_keep.add(brother) # make sure filtered out siblings don't filter out ones we want to keep
        for parent in obodag[brother].parents:
            siblings = set([y.id for y in parent.children])
            siblings.remove(brother)
            for sibling in siblings:
                if sibling in go_locus.keys() and go_locus[brother] == go_locus[sibling] and not sibling in to_keep:
                    to_remove.add(sibling)
    go_locus = {key: value for (key, value) in go_locus.items() if key not in to_remove}

    return go_locus

def gomap_to_csv(go2gene, out_file = 'test.tsv'):

    out = ""
    for term, loci in go2gene.items():
        out += "{}\t".format(term)
        out += "{}\t".format(obodag[term].name)
        out += '\t'.join(list(map(str,loci)))
        out += '\n'
        
    with open(out_file, 'w') as fout:
        fout.write(out)

In [4]:
def run_go_gsea(rank_df, g2g_map, seed, outdir='tmp'):
    """
    A ranked df and go2gene mapping returns the result dataframe for GSEA against all go-Terms
    
    loci must be grouped such that avg_ratio and p-values are correct for 1 phenotype
    see rt_unenr_grouped_loci above for example
    """
    import gseapy as gp
        
    # save the go 2 gene map, since gseapy doesn't seem to be able to use one already in memory
    gomap_to_csv(g2g_map, 'temp.gmt')
    
    res = gp.prerank(rnk=rank_df, gene_sets='temp.gmt', outdir=outdir, min_size = 5, max_size=500, 
                     permutation_n = 10000, graph_num = 20, seed=seed)
      
    def get_go_name(term):
        return obodag[term].name
    
    res['name'] = res.index.map(get_go_name)
    
    return res.sort_values('nes', ascending=False)

def plot_gsea_result(row, rank):
    return gp.plot.gsea_plot(rank, row['name'], row.hit_index, row.nes, row.pval, row.fdr, row.rank_ES, phenoPos='Tcell', phenoNeg='RAG')

In [5]:
mf_map = make_go2Gene_map(grouped_loci)
mf_map_f = filter_go2gene_map(mf_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(mf_map), len(mf_map_f)))

bp_map = make_go2Gene_map(grouped_loci, 'BP')
bp_map_f = filter_go2gene_map(bp_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(bp_map), len(bp_map_f)))

cc_map = make_go2Gene_map(grouped_loci, 'CC')
cc_map_f = filter_go2gene_map(cc_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(cc_map), len(cc_map_f)))

Unfiltered: 596	Filtered: 241
Unfiltered: 751	Filtered: 271
Unfiltered: 103	Filtered: 44


## RT vs Control Human/Mouse

In [6]:
out_dir = "RT_control_hm_gsea"
df = pd.read_csv(os.path.join(BASE,"RT_control_results_named_annot.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
#cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_control = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_control = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
#cc_rt = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
#cc_control = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-04-04 12:35:46,520 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-04-04 12:35:46,562 0005 gene_sets used for further statistical testing.....
2017-04-04 12:35:46,562 Start to run GSEA...Might take a while..................
2017-04-04 12:35:47,192 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-04-04 12:35:48,539 Congratulations...GSEAPY run successfully...............
2017-04-04 12:35:48,557 Parsing data files for GSEA.............................
2017-04-04 12:35:48,603 0010 gene_sets used for further statistical testi

In [7]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [8]:
mf_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [9]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0065007,0.576,1.796,0.007,0.042,86,14,"[165648111, 165648112, 61499238, 61499613, 61503068, 165636196, 165642139, 165646068, 165661488, 61457133, 61504270, 165661076, 61499612, 165666724]",biological regulation
GO:0050789,0.609,1.733,0.009,0.038,80,10,"[165648111, 165648112, 61499238, 61503068, 165636196, 165642139, 165646068, 61457133, 61504270, 165666724]",regulation of biological process


In [10]:
bp_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


## RT vs Control Non-Human/Mouse

In [11]:
out_dir = "RT_control_gsea"
df = pd.read_csv(os.path.join(BASE,"RT_control_results_named_annot.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[~df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)

In [12]:
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_control = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_control = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
cc_rt = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
cc_control = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-04-04 12:35:53,341 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-04-04 12:35:53,484 0054 gene_sets used for further statistical testing.....
2017-04-04 12:35:53,485 Start to run GSEA...Might take a while..................
2017-04-04 12:36:35,076 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-04-04 12:36:41,029 Congratulations...GSEAPY run successfully...............
2017-04-04 12:36:41,047 Parsing data files for GSEA.............................
2017-04-04 12:36:41,205 0063 gene_sets used for further statistical testi

In [13]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0003723,0.497,2.713,0.000e+00,0.000e+00,367,51,"[7655641, 64669875, 20323736, 33709212, 10185112, 5141220, 8283720, 17035162, 52960359, 5730052, 8974409, 23226729, 6962424, 15891127, 29630113, 23720124, 8107184, 26879329, 20132923, 16968363, 38183434, 40487328, 15916898, 5365520, 61892960, 3492858, 4100571, 10593880, 62224782, 41621220, 36697039, 61884939, 23721515, 38302786, 12493950, 30459755, 31342205, 33208200, 12690700, 10631876, 5753625, 68927583, 30556349, 65259195, 167619418, 65777235, 62322368, 10899352, 67968100, 29262834, 65669111]",RNA binding
GO:0004618,0.644,2.616,0.000e+00,5.094e-05,78,19,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4737261, 4559493, 45207415, 33420375, 22187812, 67028544, 59725586, 54913980, 4220178, 168089107, 65935692, 62829929, 63123662]",phosphoglycerate kinase activity
GO:0016774,0.582,2.472,0.000e+00,1.019e-04,111,22,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4737261, 4559493, 45207415, 33420375, 22187812, 67028544, 59725586, 54913980, 3884277, 4220178, 63123377, 168089107, 65935692, 62829929, 13604374, 63123662]","phosphotransferase activity, carboxyl group as acceptor"
GO:0001882,0.465,2.212,2.914e-04,2.623e-03,398,31,"[5888442, 80382425, 10185112, 21138590, 69205977, 5730052, 8974409, 32794085, 12661961, 40640683, 12999155, 6243639, 23375966, 26879329, 4100571, 36454371, 61884939, 61982117, 49556056, 69572327, 61106614, 15782696, 5753625, 30556349, 80394470, 67719393, 15769577, 167619418, 40723118, 29262834, 3557492]",nucleoside binding
GO:0016301,0.443,2.174,2.895e-04,3.199e-03,480,34,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4737261, 4559493, 45207415, 33420375, 22187812, 61635847, 67028544, 59725586, 54913980, 3884277, 17583418, 61708790, 4220178, 62102911, 63123377, 51371961, 36117032, 62942343, 48698283, 168089107, 65935692, 55261886, 48298182, 64346968, 62829929, 13604374, 62098267, 63123662]",kinase activity
GO:0019001,0.449,2.064,1.937e-03,7.810e-03,248,28,"[5888442, 80382425, 10185112, 21138590, 69205977, 5730052, 8974409, 32794085, 12661961, 40640683, 26879329, 4100571, 36454371, 61884939, 61982117, 49556056, 69572327, 61106614, 15782696, 5753625, 30556349, 80394470, 67719393, 15769577, 167619418, 40723118, 29262834, 3557492]",guanyl nucleotide binding
GO:0001883,0.449,2.062,1.185e-03,6.782e-03,248,28,"[5888442, 80382425, 10185112, 21138590, 69205977, 5730052, 8974409, 32794085, 12661961, 40640683, 26879329, 4100571, 36454371, 61884939, 61982117, 49556056, 69572327, 61106614, 15782696, 5753625, 30556349, 80394470, 67719393, 15769577, 167619418, 40723118, 29262834, 3557492]",purine nucleoside binding
GO:0003735,0.334,1.963,1.743e-03,1.406e-02,360,65,"[7655641, 64669875, 20323736, 33709212, 19107849, 29630217, 5141220, 8283720, 17035162, 5645650, 60453073, 165941560, 28829095, 25902959, 20862748, 23226729, 6962424, 15891127, 29630113, 23720124, 6288540, 8107184, 20132923, 16968363, 38183434, 40487328, 15916898, 5365520, 3492858, 10593880, 62224782, 41621220, 36697039, 23449836, 23721515, 38302786, 12493950, 30459755, 31342205, 33208200, 12690700, 3576806, 68927583, 65259195, 65777235, 67911420, 39007615, 10899352, 64474525, 42272589, 66752541, 167990362, 68373997, 65506212, 65977068, 67968100, 33140401, 167623679, 65626218, 67383571, 38249992, 10276368, 65669111, 13581456, 166036375]",structural constituent of ribosome
GO:0022892,0.504,1.875,6.516e-03,2.511e-02,89,15,"[3885199, 1994037, 29978920, 33721585, 38210955, 54503535, 21357752, 32363526, 1809293, 268176, 21432496, 27807827, 62660497, 64582513, 165706474]",substrate-specific transporter activity


In [14]:
mf_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0004553,-0.480,-1.767,0.018,0.345,75,11,"[11310495, 11310502, 167639021, 61777171, 63376055, 63922748, 167549118, 62488243, 62521472, 13605039, 62450924]","hydrolase activity, hydrolyzing O-glycosyl compounds"
GO:0016798,-0.480,-1.765,0.019,0.175,76,11,"[11310495, 11310502, 167639021, 61777171, 63376055, 63922748, 167549118, 62488243, 62521472, 13605039, 62450924]","hydrolase activity, acting on glycosyl bonds"
GO:0050662,-0.328,-1.734,0.017,0.138,235,27,"[29001612, 27290903, 26904631, 29983272, 20853009, 7496609, 6997781, 3740901, 15860346, 63640418, 20839749, 63296313, 63305114, 63379850, 62942791, 168109762, 63626599, 63235820, 61621955, 63648739, 63662802, 63389326, 61809381, 48540155, 21899103, 61608244, 166278783]",coenzyme binding
GO:0009055,-0.541,-1.733,0.027,0.104,38,8,"[63305114, 63379850, 62942791, 49911241, 63626599, 61957888, 63389326, 167701862]",electron carrier activity
GO:0016903,-0.366,-1.589,0.048,0.173,282,16,"[36107728, 6997781, 43414939, 3740901, 15860346, 10318713, 63640418, 61880488, 168109762, 61621955, 45723591, 62312164, 57098781, 21899103, 61608244, 166278783]","oxidoreductase activity, acting on the aldehyde or oxo group of donors"


In [15]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0009132,0.408,2.117,8.412e-04,0.033,304,41,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4737261, 4559493, 45207415, 29629717, 33420375, 9654148, 23089117, 25933508, 22187812, 63441836, 67028544, 59725586, 54913980, 30623941, 839701, 12964871, 17046519, 5960044, 8200910, 65033516, 30588025, 64427981, 4220178, 62102911, 168089107, 65935692, 55261886, 48298182, 64346968, 62829929, 62566960, 64479173, 64978197, 62098267, 63123662]",nucleoside diphosphate metabolic process
GO:0046939,0.408,2.113,8.461e-04,0.017,304,41,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4737261, 4559493, 45207415, 29629717, 33420375, 9654148, 23089117, 25933508, 22187812, 63441836, 67028544, 59725586, 54913980, 30623941, 839701, 12964871, 17046519, 5960044, 8200910, 65033516, 30588025, 64427981, 4220178, 62102911, 168089107, 65935692, 55261886, 48298182, 64346968, 62829929, 62566960, 64479173, 64978197, 62098267, 63123662]",nucleotide phosphorylation
GO:0009135,0.408,2.111,9.901e-04,0.012,302,41,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4737261, 4559493, 45207415, 29629717, 33420375, 9654148, 23089117, 25933508, 22187812, 63441836, 67028544, 59725586, 54913980, 30623941, 839701, 12964871, 17046519, 5960044, 8200910, 65033516, 30588025, 64427981, 4220178, 62102911, 168089107, 65935692, 55261886, 48298182, 64346968, 62829929, 62566960, 64479173, 64978197, 62098267, 63123662]",purine nucleoside diphosphate metabolic process
GO:0072524,0.401,2.091,7.072e-04,0.010,305,42,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4737261, 4559493, 45207415, 29629717, 33420375, 9654148, 23089117, 25933508, 22187812, 63441836, 67028544, 59725586, 54913980, 30623941, 839701, 12964871, 17046519, 5960044, 8200910, 65033516, 30588025, 64427981, 4220178, 62102911, 168089107, 65935692, 166574781, 55261886, 48298182, 64346968, 62829929, 62566960, 64479173, 64978197, 62098267, 63123662]",pyridine-containing compound metabolic process
GO:0006733,0.401,2.089,7.050e-04,0.008,305,42,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4737261, 4559493, 45207415, 29629717, 33420375, 9654148, 23089117, 25933508, 22187812, 63441836, 67028544, 59725586, 54913980, 30623941, 839701, 12964871, 17046519, 5960044, 8200910, 65033516, 30588025, 64427981, 4220178, 62102911, 168089107, 65935692, 166574781, 55261886, 48298182, 64346968, 62829929, 62566960, 64479173, 64978197, 62098267, 63123662]",oxidoreduction coenzyme metabolic process
GO:0044267,0.352,2.088,0.000e+00,0.007,413,69,"[7655641, 64669875, 20323736, 33709212, 27212193, 38310093, 19107849, 29630217, 5141220, 8283720, 17035162, 5645650, 60453073, 165941560, 28829095, 25902959, 20862748, 23226729, 6962424, 15891127, 29630113, 23720124, 6288540, 8107184, 20132923, 16968363, 38183434, 40487328, 15916898, 5365520, 3492858, 21157792, 10593880, 62224782, 41621220, 36697039, 23449836, 23721515, 38302786, 12493950, 30459755, 31342205, 33208200, 12690700, 3576806, 68927583, 65259195, 28150928, 65777235, 67911420, 39007615, 10899352, 64474525, 42272589, 66752541, 167990362, 68373997, 65506212, 65977068, 67968100, 33140401, 167623679, 65626218, 67383571, 38249992, 10276368, 65669111, 13581456, 166036375]",cellular protein metabolic process
GO:0009141,0.392,2.087,5.639e-04,0.006,381,45,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4737261, 4559493, 45207415, 29629717, 33420375, 1994037, 9654148, 23089117, 25933508, 32363526, 1809293, 22187812, 268176, 63441836, 67028544, 59725586, 54913980, 30623941, 839701, 12964871, 17046519, 5960044, 8200910, 65033516, 30588025, 64427981, 4220178, 62102911, 168089107, 65935692, 55261886, 48298182, 64346968, 62829929, 62566960, 64479173, 64978197, 62098267, 63123662]",nucleoside triphosphate metabolic process
GO:0046034,0.392,2.086,1.533e-03,0.005,379,45,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4737261, 4559493, 45207415, 29629717, 33420375, 1994037, 965

In [16]:
bp_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [17]:
cc_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0015934,0.660,2.114,6.653e-04,0.012,63,10,"[7655641, 64669875, 20323736, 33709212, 8283720, 17035162, 15891127, 41621220, 10899352, 38249992]",large ribosomal subunit
GO:0044444,0.335,2.014,1.710e-03,0.015,422,70,"[7655641, 64669875, 20323736, 33709212, 29629717, 19107849, 29630217, 5141220, 8283720, 17035162, 23089117, 5645650, 60453073, 165941560, 28829095, 25902959, 20862748, 23226729, 6962424, 15891127, 29630113, 23720124, 6288540, 8107184, 20132923, 16968363, 38183434, 40487328, 15916898, 5365520, 3492858, 80889105, 10593880, 62224782, 41621220, 36697039, 23449836, 23721515, 38302786, 12493950, 30459755, 31342205, 33208200, 12690700, 3576806, 68927583, 65259195, 65777235, 67911420, 39007615, 10899352, 64474525, 42272589, 66752541, 167990362, 68373997, 65506212, 65977068, 67968100, 33140401, 61809381, 167623679, 65626218, 62566960, 67383571, 38249992, 10276368, 65669111, 13581456, 166036375]",cytoplasmic part
GO:0044446,0.491,1.988,2.516e-03,0.013,169,19,"[7655641, 64669875, 20323736, 33709212, 8283720, 17035162, 23226729, 6962424, 15891127, 6288540, 20132923, 3492858, 10593880, 41621220, 68927583, 65259195, 10899352, 38249992, 65669111]",intracellular organelle part
GO:0044391,0.491,1.987,1.997e-03,0.010,159,19,"[7655641, 64669875, 20323736, 33709212, 8283720, 17035162, 23226729, 6962424, 15891127, 6288540, 20132923, 3492858, 10593880, 41621220, 68927583, 65259195, 10899352, 38249992, 65669111]",ribosomal subunit
GO:1990904,0.334,1.952,2.281e-03,0.010,360,65,"[7655641, 64669875, 20323736, 33709212, 19107849, 29630217, 5141220, 8283720, 17035162, 5645650, 60453073, 165941560, 28829095, 25902959, 20862748, 23226729, 6962424, 15891127, 29630113, 23720124, 6288540, 8107184, 20132923, 16968363, 38183434, 40487328, 15916898, 5365520, 3492858, 10593880, 62224782, 41621220, 36697039, 23449836, 23721515, 38302786, 12493950, 30459755, 31342205, 33208200, 12690700, 3576806, 68927583, 65259195, 65777235, 67911420, 39007615, 10899352, 64474525, 42272589, 66752541, 167990362, 68373997, 65506212, 65977068, 67968100, 33140401, 167623679, 65626218, 67383571, 38249992, 10276368, 65669111, 13581456, 166036375]",ribonucleoprotein complex
GO:0005622,0.377,1.932,3.536e-03,0.010,306,41,"[7655641, 64669875, 20323736, 33709212, 19107849, 29630217, 10185112, 5141220, 5645650, 60453073, 5730052, 8974409, 165941560, 28829095, 25902959, 20862748, 15891127, 26879329, 4100571, 41621220, 3884277, 23449836, 61884939, 3576806, 5753625, 30556349, 65259195, 167619418, 63123377, 10899352, 64474525, 42272589, 68373997, 67968100, 33140401, 67383571, 38249992, 29262834, 10276368, 65669111, 13604374]",intracellular
GO:0043232,0.312,1.764,1.199e-02,0.031,320,57,"[7655641, 64669875, 20323736, 33709212, 19107849, 29630217, 5141220, 5645650, 60453073, 165941560, 28829095, 25902959, 20862748, 15891127, 29630113, 23720124, 8107184, 16968363, 38183434, 40487328, 15916898, 5365520, 3492858, 10593880, 62224782, 41621220, 36697039, 23449836, 23721515, 38302786, 12493950, 30459755, 31342205, 33208200, 12690700, 3576806, 68927583, 65777235, 67911420, 39007615, 10899352, 64474525, 42272589, 66752541, 167990362, 68373997, 65506212, 65977068, 67968100, 33140401, 167623679, 65626218, 67383571, 38249992, 10276368, 13581456, 166036375]",intracellular non-membrane-bounded organelle
GO:0005840,0.312,1.762,1.024e-02,0.028,319,57,"[7655641, 64669875, 20323736, 33709212, 19107849, 29630217, 5141220, 5645650, 60453073, 165941560, 28829095, 25902959, 20862748, 15891127, 29630113, 23720124, 8107184, 16968363, 38183434, 40487328, 15916898, 5365520, 3492858, 10593880, 62224782, 41621220, 36697039, 23449836, 23721515, 38302786, 12493950, 30459755, 31342205, 33208200, 12690700, 3576806, 68927583, 65777235, 67911420, 39007615, 10899352, 64474525, 42272589, 66752541, 167990362, 68373997, 65506212, 65977068, 67968100, 33140401, 167623679, 65626218, 67383571, 38249992, 10276368, 13581456, 166036375]",ribosome
GO:0043229,0.312

In [18]:
cc_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0016020,-0.466,-1.946,0.007,0.043,75,15,"[21357752, 21687303, 167156307, 61699598, 166632550, 64013758, 61660751, 62830069, 62660497, 61957888, 166216926, 166593522, 167701862, 27635679, 165685567]",membrane
GO:0043190,-0.527,-1.786,0.021,0.061,115,9,"[63132923, 47513944, 11992086, 56076521, 17618404, 10265600, 37969849, 63374013, 28273397]",ATP-binding cassette (ABC) transporter complex
GO:0098797,-0.527,-1.770,0.018,0.045,117,9,"[63132923, 47513944, 11992086, 56076521, 17618404, 10265600, 37969849, 63374013, 28273397]",plasma membrane protein complex
GO:0044459,-0.527,-1.770,0.024,0.034,119,9,"[63132923, 47513944, 11992086, 56076521, 17618404, 10265600, 37969849, 63374013, 28273397]",plasma membrane part
GO:1904949,-0.527,-1.767,0.019,0.027,117,9,"[63132923, 47513944, 11992086, 56076521, 17618404, 10265600, 37969849, 63374013, 28273397]",ATPase complex
GO:1990351,-0.527,-1.766,0.021,0.023,117,9,"[63132923, 47513944, 11992086, 56076521, 17618404, 10265600, 37969849, 63374013, 28273397]",transporter complex


## Rag vs WT Human/Mouse

In [19]:
out_dir = "Rag_WT_hm_gsea"
df = pd.read_csv(os.path.join(BASE,"Rag_WT_results_named_annot.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
#cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rag = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_wt = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rag = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_wt = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
#cc_rt = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
#cc_control = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-04-04 12:37:54,857 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-04-04 12:37:54,889 0004 gene_sets used for further statistical testing.....
2017-04-04 12:37:54,890 Start to run GSEA...Might take a while..................
2017-04-04 12:37:55,244 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-04-04 12:37:56,299 Congratulations...GSEAPY run successfully...............
2017-04-04 12:37:56,315 Parsing data files for GSEA.............................
2017-04-04 12:37:56,354 0004 gene_sets used for further statistical testi

In [20]:
mf_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0046872,0.531,1.679,0.028,0.087,488,9,"[61501076, 61502914, 61500363, 61501312, 61499612, 165666724, 61499523, 61495726, 165637224]",metal ion binding


In [21]:
mf_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0005515,-0.864,-1.869,1.363e-04,6.931e-04,108,6,"[165662997, 61499944, 165642270, 165664161, 165646313, 165661842]",protein binding


In [22]:
bp_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [23]:
bp_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


## Rag vs WT Non-Human/Mouse

In [24]:
out_dir = "Rag_WT_gsea"
df = pd.read_csv(os.path.join(BASE,"Rag_WT_results_named_annot.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[~df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)

In [25]:
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rag = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_wt = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rag = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_wt = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
cc_rag = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
cc_wt = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-04-04 12:37:58,412 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-04-04 12:37:58,798 0074 gene_sets used for further statistical testing.....
2017-04-04 12:37:58,799 Start to run GSEA...Might take a while..................
2017-04-04 12:40:36,547 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-04-04 12:40:42,654 Congratulations...GSEAPY run successfully...............
2017-04-04 12:40:42,670 Parsing data files for GSEA.............................
2017-04-04 12:40:43,051 0103 gene_sets used for further statistical testi

In [26]:
mf_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0016903,0.427,2.458,0.000e+00,1.541e-04,282,75,"[57377692, 57602627, 20987406, 10318713, 61599909, 63509022, 6997781, 166781730, 44171740, 61901179, 64549605, 61599853, 61601993, 61717636, 61880488, 53586382, 63603065, 39787404, 63849179, 166052469, 50431192, 166155431, 61033726, 61686088, 6574706, 61986126, 52336699, 61603708, 63571991, 57894284, 58687019, 62139539, 52783963, 168109762, 59329815, 63900260, 61600015, 61963309, 61600538, 166304100, 64211477, 66913607, 60255223, 44281203, 62141702, 41869870, 63007267, 52484917, 64109059, 53437685, 55526608, 47654592, 37964574, 10739365, 50135248, 64872068, 167530105, 62669104, 49924705, 62232545, 20607209, 62943474, 61598699, 63032818, 61588102, 57566721, 62151332, 45794851, 40525078, 54342385, 45816831, 53596244, 63371685, 40636025, 63600850]","oxidoreductase activity, acting on the aldehyde or oxo group of donors"
GO:0009055,0.730,2.457,0.000e+00,7.706e-05,38,12,"[63098496, 64646230, 21834593, 6263025, 63079626, 62114668, 11316432, 47763199, 11316431, 167701862, 64864375, 63507869]",electron carrier activity
GO:0051540,0.479,2.329,0.000e+00,4.110e-04,149,38,"[62751142, 63901915, 167744672, 62114668, 28421698, 57377692, 57602627, 20987406, 61599909, 63509022, 64549605, 61599853, 61601993, 167701862, 39787404, 166052469, 61603708, 58687019, 62139539, 168109762, 61600015, 61963309, 61600538, 166304100, 64211477, 27694381, 60255223, 62141702, 64109059, 50135248, 49924705, 20607209, 61598699, 61588102, 62151332, 45794851, 54342385, 45816831]",metal cluster binding
GO:0016820,0.460,2.183,1.898e-04,2.138e-03,190,35,"[62344870, 168161151, 28240379, 40674602, 62545344, 62072713, 21258712, 165851092, 63132923, 65516796, 66978191, 31357673, 32955361, 40686252, 13203808, 32157942, 13841186, 6347838, 12378604, 8187004, 62946388, 6934942, 47513944, 37969849, 63199719, 10732421, 27603383, 62055617, 33383613, 62354250, 65627086, 63129880, 61865720, 63078357, 13568664]","hydrolase activity, acting on acid anhydrides, catalyzing transmembrane movement of substances"
GO:0016868,0.613,2.061,1.738e-03,5.379e-03,79,12,"[167401290, 39279903, 62071220, 168126288, 41534199, 8102980, 53821637, 38613059, 67730070, 62143029, 42049385, 62198791]","intramolecular transferase activity, phosphotransferases"
GO:0016866,0.613,2.041,2.309e-03,5.381e-03,112,12,"[167401290, 39279903, 62071220, 168126288, 41534199, 8102980, 53821637, 38613059, 67730070, 62143029, 42049385, 62198791]",intramolecular transferase activity
GO:0022804,0.426,2.033,1.715e-03,5.075e-03,205,36,"[62344870, 168161151, 28240379, 40674602, 62545344, 62072713, 21258712, 165851092, 63132923, 65516796, 66978191, 31357673, 32955361, 40686252, 13203808, 32157942, 13841186, 6347838, 12378604, 8187004, 62946388, 6934942, 47513944, 37969849, 63199719, 10732421, 27603383, 62055617, 33383613, 62354250, 65627086, 63129880, 61865720, 63078357, 165706474, 13568664]",active transmembrane transporter activity
GO:0016620,0.433,1.951,1.539e-03,9.536e-03,101,30,"[10318713, 6997781, 166781730, 44171740, 61717636, 53586382, 50431192, 166155431, 61033726, 61686088, 6574706, 61986126, 52336699, 63571991, 57894284, 52783963, 66913607, 44281203, 41869870, 63007267, 52484917, 53437685, 55526608, 47654592, 10739365, 167530105, 62943474, 63032818, 63371685, 63600850]","oxidoreductase activity, acting on the aldehyde or oxo group of donors, NAD or NADP as acceptor"
GO:0022857,0.384,1.849,4.042e-03,1.968e-02,212,37,"[62344870, 168161151, 28240379, 40674602, 62545344, 62072713, 21258712, 165851092, 63132923, 65516796, 66978191, 31357673, 32955361, 40686252, 13203808, 32157942, 13841186, 6347838, 12378604, 8187004, 62946388, 6934942, 47513944, 37969849, 63199719, 10732421, 27603383, 62055617, 33383613, 62354250, 65627086, 63129880, 61865720, 63078357, 165706474, 64582513, 13568664]",transmembrane transporter activity


In [27]:
mf_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0097747,-0.653,-4.453,0.000e+00,0.000e+00,345,143,"[61571198, 61950469, 47461425, 15059448, 8291718, 17558356, 7937195, 15768925, 9047445, 6340712, 10774345, 10795597, 20809021, 48701865, 51091566, 41242615, 37208030, 42830626, 21296655, 61588900, 60772420, 56745534, 50065172, 62167253, 44186767, 167412647, 61599553, 17559280, 30840365, 6056719, 6023174, 14700148, 10699458, 7440434, 5830788, 16830531, 36374512, 12803565, 22055143, 27851830, 166882954, 43146673, 21868198, 61079203, 61952789, 7691951, 21252862, 50058218, 27880167, 61604840, 22236358, 61597374, 61579823, 45428002, 58926833, 63555859, 62326869, 39588863, 61605755, 40887919, 61597571, 61601524, 14167279, 24340641, 21966233, 21075844, 61580012, 61580937, 18214391, 9670834, 61573275, 8305836, 61669421, 61603176, 61601759, 7435501, 56735007, 65613889, 63632220, 58090628, 30077373, 26013941, 44284503, 66863306, 63185291, 41171397, 45190220, 21909748, 63027032, 62634255, 20706901, 62427434, 58137973, 62092597, 59642867, 37935841, 62130826, 38315748, 58701908, 41204377, ...]",RNA polymerase activity
GO:0016779,-0.614,-4.359,0.000e+00,0.000e+00,463,171,"[18523127, 28436439, 40551040, 59156012, 63039326, 57028376, 65657046, 63112646, 38175368, 62718684, 27804972, 61571198, 61950469, 47461425, 15059448, 8291718, 17558356, 7937195, 15768925, 9047445, 6340712, 10774345, 10795597, 20809021, 48701865, 51091566, 41242615, 37208030, 42830626, 21296655, 61588900, 60772420, 56745534, 50065172, 62167253, 44186767, 167277977, 167412647, 61599553, 61793630, 17559280, 30840365, 6056719, 6023174, 14700148, 10699458, 7440434, 5830788, 16830531, 36374512, 12803565, 22055143, 27851830, 166882954, 43146673, 21868198, 44355835, 61079203, 49566268, 43390620, 48648504, 56494061, 41923576, 61835191, 63032743, 50389668, 43556762, 61952789, 10515859, 63411294, 61787527, 7691951, 21252862, 50058218, 61780119, 27880167, 61604840, 22236358, 61597374, 61579823, 45428002, 58926833, 63555859, 62326869, 39588863, 61605755, 40887919, 61597571, 61601524, 14167279, 24340641, 21966233, 21075844, 61580012, 61580937, 18214391, 9670834, 61573275, 8305836, 61669421, ...]",nucleotidyltransferase activity
GO:0003677,-0.622,-4.303,0.000e+00,0.000e+00,369,155,"[66926567, 68934118, 51907206, 56510025, 68645600, 46260937, 61571198, 61950469, 47461425, 15059448, 8291718, 17558356, 7937195, 15768925, 9047445, 6340712, 10774345, 10795597, 20809021, 48701865, 51091566, 41242615, 37208030, 42830626, 21296655, 61588900, 60772420, 56745534, 50065172, 62167253, 44186767, 167412647, 61599553, 17559280, 30840365, 6056719, 6023174, 14700148, 10699458, 7440434, 5830788, 16830531, 36374512, 12803565, 22055143, 27851830, 166882954, 43146673, 21868198, 61079203, 61952789, 7691951, 21252862, 50058218, 27880167, 61604840, 22236358, 61597374, 61579823, 45428002, 58926833, 63555859, 62326869, 39588863, 61605755, 40887919, 61597571, 61601524, 14167279, 24340641, 21966233, 21075844, 61580012, 61580937, 18214391, 9670834, 61573275, 8305836, 61669421, 61603176, 61601759, 7435501, 56735007, 65613889, 63632220, 58090628, 30077373, 26013941, 44284503, 66863306, 63185291, 41171397, 45190220, 21909748, 63027032, 62634255, 20706901, 62427434, 58137973, 62092597, ...]",DNA binding
GO:0004634,-0.870,-3.487,0.000e+00,0.000e+00,43,19,"[62743325, 62658191, 22213675, 18120033, 66797374, 62783504, 3624425, 20853105, 10876818, 45981629, 46324053, 62809084, 21126143, 5189919, 20745054, 66137094, 31709972, 62848403, 17549174]",phosphopyruvate hydratase activity
GO:0000287,-0.669,-3.414,0.000e+00,0.000e+00,113,42,"[39279903, 62071220, 168126288, 41534199, 8102980, 53821637, 38613059, 62743325, 62143029, 42049385, 62198791, 8551124, 62658191, 62737033, 64587749, 63158778, 62853917, 63677227, 10822577, 54615426, 61965999, 21360480, 22213675, 18120033, 66797374, 62783504, 62289912, 65218941, 3624425, 20853105, 28234013, 10876818, 45981629, 46324053, 62809084, 21126143, 5189919, 20745054, 6

In [28]:
bp_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0040011,0.543,3.224,0.000e+00,0.000,274,85,"[63837786, 165990924, 167953213, 168171488, 167966912, 13586931, 165936696, 62136808, 167570053, 64855697, 64780867, 62417891, 62405618, 62303301, 42831732, 62719918, 62849941, 168192733, 62771320, 62619244, 62923262, 62926584, 63144665, 28438217, 62002315, 165775316, 57751592, 63195134, 66593668, 63842059, 63275290, 64065097, 167641590, 13574239, 48437829, 52127537, 68252636, 167628538, 166007447, 167438835, 62565570, 62396394, 166874224, 167434350, 62085029, 13589494, 65988434, 64297496, 167832826, 167343151, 13586933, 167738934, 168200142, 66403957, 11407241, 44091081, 167337265, 11413856, 22310619, 167047560, 63798592, 10082333, 63500843, 63295556, 166530730, 67398658, 168112544, 28430116, 62300168, 11548874, 168001411, 166818826, 64573539, 63489079, 63911288, 63937175, 66399802, 64842974, 165887166, 166824805, 166695246, 62626786, 167344702, 167799665, 166636440]",locomotion
GO:0006928,0.534,3.148,0.000e+00,0.000,270,81,"[167953213, 168171488, 167966912, 13586931, 62136808, 167570053, 64855697, 64780867, 62417891, 62405618, 62303301, 42831732, 62719918, 62849941, 168192733, 62771320, 62619244, 62923262, 62926584, 63144665, 28438217, 62002315, 165775316, 57751592, 63195134, 66593668, 63842059, 63275290, 64065097, 167641590, 13574239, 48437829, 52127537, 68252636, 167628538, 166007447, 167438835, 62565570, 62396394, 166874224, 167434350, 62085029, 13589494, 65988434, 64297496, 167832826, 167343151, 13586933, 167738934, 168200142, 66403957, 11407241, 167337265, 11413856, 22310619, 167047560, 63798592, 10082333, 63500843, 63295556, 166530730, 67398658, 168112544, 28430116, 62300168, 11548874, 168001411, 166818826, 64573539, 63489079, 63911288, 63937175, 66399802, 64842974, 165887166, 166824805, 166695246, 62626786, 167344702, 167799665, 166636440]",movement of cell or subcellular component
GO:0048870,0.534,3.142,0.000e+00,0.000,269,81,"[167953213, 168171488, 167966912, 13586931, 62136808, 167570053, 64855697, 64780867, 62417891, 62405618, 62303301, 42831732, 62719918, 62849941, 168192733, 62771320, 62619244, 62923262, 62926584, 63144665, 28438217, 62002315, 165775316, 57751592, 63195134, 66593668, 63842059, 63275290, 64065097, 167641590, 13574239, 48437829, 52127537, 68252636, 167628538, 166007447, 167438835, 62565570, 62396394, 166874224, 167434350, 62085029, 13589494, 65988434, 64297496, 167832826, 167343151, 13586933, 167738934, 168200142, 66403957, 11407241, 167337265, 11413856, 22310619, 167047560, 63798592, 10082333, 63500843, 63295556, 166530730, 67398658, 168112544, 28430116, 62300168, 11548874, 168001411, 166818826, 64573539, 63489079, 63911288, 63937175, 66399802, 64842974, 165887166, 166824805, 166695246, 62626786, 167344702, 167799665, 166636440]",cell motility
GO:0042558,0.648,2.104,5.870e-04,0.007,40,11,"[31379006, 61964112, 62018486, 65135677, 63995561, 17571629, 51305083, 64709852, 67213458, 64164012, 165831755]",pteridine-containing compound metabolic process
GO:0042559,0.640,2.002,2.145e-03,0.014,29,10,"[61964112, 62018486, 65135677, 63995561, 17571629, 51305083, 64709852, 67213458, 64164012, 165831755]",pteridine-containing compound biosynthetic process
GO:0006575,0.640,1.988,2.322e-03,0.013,30,10,"[61964112, 62018486, 65135677, 63995561, 17571629, 51305083, 64709852, 67213458, 64164012, 165831755]",cellular modified amino acid metabolic process
GO:1901605,0.438,1.807,8.340e-03,0.049,110,22,"[63889213, 165956827, 63092817, 166239197, 27696183, 21808882, 63863560, 62743829, 166756527, 61703706, 65619550, 63323004, 37425343, 166114562, 20611399, 62207461, 22083024, 62840735, 62652073, 63437396, 63600850, 9720195]",alpha-amino acid metabolic process
GO:0051234,0.328,1.796,3.290e-03,0.047,331,61,"[167285315, 13587543, 167348034, 165992613, 167399046, 62160260, 21799059, 62047389, 62623865, 166635414, 49707019, 168124343, 11412988, 62344870, 28423332, 168161151, 63070669, 166208277, 28240379, 406746

In [29]:
bp_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0032774,-0.653,-4.449,0.000e+00,0.000e+00,345,143,"[61571198, 61950469, 47461425, 15059448, 8291718, 17558356, 7937195, 15768925, 9047445, 6340712, 10774345, 10795597, 20809021, 48701865, 51091566, 41242615, 37208030, 42830626, 21296655, 61588900, 60772420, 56745534, 50065172, 62167253, 44186767, 167412647, 61599553, 17559280, 30840365, 6056719, 6023174, 14700148, 10699458, 7440434, 5830788, 16830531, 36374512, 12803565, 22055143, 27851830, 166882954, 43146673, 21868198, 61079203, 61952789, 7691951, 21252862, 50058218, 27880167, 61604840, 22236358, 61597374, 61579823, 45428002, 58926833, 63555859, 62326869, 39588863, 61605755, 40887919, 61597571, 61601524, 14167279, 24340641, 21966233, 21075844, 61580012, 61580937, 18214391, 9670834, 61573275, 8305836, 61669421, 61603176, 61601759, 7435501, 56735007, 65613889, 63632220, 58090628, 30077373, 26013941, 44284503, 66863306, 63185291, 41171397, 45190220, 21909748, 63027032, 62634255, 20706901, 62427434, 58137973, 62092597, 59642867, 37935841, 62130826, 38315748, 58701908, 41204377, ...]",RNA biosynthetic process
GO:0072350,-0.796,-3.228,0.000e+00,0.000e+00,29,20,"[38181119, 62737033, 40713606, 38860663, 61303090, 53916053, 64587749, 63158778, 11427336, 62853917, 168067151, 63677227, 10822577, 54615426, 61965999, 20822172, 21360480, 50196479, 38164996, 53819948]",tricarboxylic acid metabolic process
GO:0009259,-0.498,-2.877,0.000e+00,0.000e+00,414,66,"[167029093, 167526299, 165866251, 62344870, 64711323, 63185681, 166165052, 63157973, 18134535, 64380885, 53022688, 40688875, 62072713, 20752463, 64530918, 69926951, 64182902, 63865771, 62743325, 62661009, 18128369, 47619855, 45919266, 32079045, 62560655, 166007049, 62354250, 54193303, 65627086, 18213545, 63272810, 8551124, 63649356, 38666650, 63517822, 62446788, 62658191, 63973174, 62636935, 22197670, 50712297, 65812454, 166287375, 22213675, 18120033, 66797374, 13568664, 62783504, 167057179, 64066327, 65218941, 3624425, 20853105, 28234013, 6131330, 10876818, 45981629, 46324053, 62809084, 21126143, 5189919, 20745054, 66137094, 31709972, 62848403, 17549174]",ribonucleotide metabolic process
GO:0009123,-0.498,-2.875,0.000e+00,0.000e+00,412,66,"[167029093, 167526299, 165866251, 62344870, 64711323, 63185681, 166165052, 63157973, 18134535, 64380885, 53022688, 40688875, 62072713, 20752463, 64530918, 69926951, 64182902, 63865771, 62743325, 62661009, 18128369, 47619855, 45919266, 32079045, 62560655, 166007049, 62354250, 54193303, 65627086, 18213545, 63272810, 8551124, 63649356, 38666650, 63517822, 62446788, 62658191, 63973174, 62636935, 22197670, 50712297, 65812454, 166287375, 22213675, 18120033, 66797374, 13568664, 62783504, 167057179, 64066327, 65218941, 3624425, 20853105, 28234013, 6131330, 10876818, 45981629, 46324053, 62809084, 21126143, 5189919, 20745054, 66137094, 31709972, 62848403, 17549174]",nucleoside monophosphate metabolic process
GO:0019693,-0.486,-2.818,0.000e+00,0.000e+00,416,67,"[167029093, 167526299, 63863560, 165866251, 62344870, 64711323, 63185681, 166165052, 63157973, 18134535, 64380885, 53022688, 40688875, 62072713, 20752463, 64530918, 69926951, 64182902, 63865771, 62743325, 62661009, 18128369, 47619855, 45919266, 32079045, 62560655, 166007049, 62354250, 54193303, 65627086, 18213545, 63272810, 8551124, 63649356, 38666650, 63517822, 62446788, 62658191, 63973174, 62636935, 22197670, 50712297, 65812454, 166287375, 22213675, 18120033, 66797374, 13568664, 62783504, 167057179, 64066327, 65218941, 3624425, 20853105, 28234013, 6131330, 10876818, 45981629, 46324053, 62809084, 21126143, 5189919, 20745054, 66137094, 31709972, 62848403, 17549174]",ribose phosphate metabolic process
GO:0009132,-0.498,-2.788,0.000e+00,0.000e+00,304,58,"[167029093, 167526299, 165866251, 64711323, 63185681, 166165052, 63157973, 18134535, 64380885, 53022688, 40688875, 20752463, 69926951, 64182902, 63865771, 62743325, 62661009, 18128369, 47619855, 45919266, 32079045, 62560655, 166007049, 

In [30]:
cc_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0042995,0.534,3.138,0.000e+00,0.000e+00,269,81,"[167953213, 168171488, 167966912, 13586931, 62136808, 167570053, 64855697, 64780867, 62417891, 62405618, 62303301, 42831732, 62719918, 62849941, 168192733, 62771320, 62619244, 62923262, 62926584, 63144665, 28438217, 62002315, 165775316, 57751592, 63195134, 66593668, 63842059, 63275290, 64065097, 167641590, 13574239, 48437829, 52127537, 68252636, 167628538, 166007447, 167438835, 62565570, 62396394, 166874224, 167434350, 62085029, 13589494, 65988434, 64297496, 167832826, 167343151, 13586933, 167738934, 168200142, 66403957, 11407241, 167337265, 11413856, 22310619, 167047560, 63798592, 10082333, 63500843, 63295556, 166530730, 67398658, 168112544, 28430116, 62300168, 11548874, 168001411, 166818826, 64573539, 63489079, 63911288, 63937175, 66399802, 64842974, 165887166, 166824805, 166695246, 62626786, 167344702, 167799665, 166636440]",cell projection
GO:0044463,0.529,2.985,0.000e+00,0.000e+00,229,66,"[167953213, 168171488, 167966912, 13586931, 62136808, 167570053, 64855697, 64780867, 62417891, 62405618, 62303301, 42831732, 62719918, 62849941, 168192733, 62771320, 62619244, 62923262, 62926584, 63144665, 28438217, 62002315, 13574239, 167628538, 167438835, 166874224, 167434350, 62085029, 13589494, 65988434, 64297496, 167832826, 167343151, 13586933, 167738934, 168200142, 66403957, 11407241, 167337265, 11413856, 22310619, 167047560, 63798592, 10082333, 63500843, 63295556, 166530730, 67398658, 168112544, 28430116, 62300168, 11548874, 168001411, 166818826, 64573539, 63489079, 63911288, 63937175, 66399802, 64842974, 165887166, 166824805, 166695246, 62626786, 167344702, 167799665]",cell projection part
GO:0044422,0.492,2.964,0.000e+00,0.000e+00,402,90,"[167953213, 168171488, 167966912, 13586931, 62136808, 167570053, 64855697, 64780867, 62417891, 62405618, 62303301, 42831732, 62719918, 62849941, 168192733, 62771320, 62619244, 62923262, 62926584, 63144665, 28438217, 62002315, 13574239, 167628538, 65259195, 167438835, 166874224, 167434350, 62085029, 13589494, 44182220, 65988434, 64297496, 167832826, 167343151, 13586933, 167738934, 168200142, 167340047, 48590768, 168017014, 29341916, 7259645, 67389484, 82054302, 166807349, 74406712, 67061811, 66403957, 166720030, 20720618, 11407241, 167337265, 66350717, 11413856, 22310619, 167047560, 65886657, 66541818, 4774708, 63798592, 10082333, 66050469, 63500843, 63295556, 166530730, 67398658, 168112544, 28430116, 62300168, 11548874, 168001411, 166818826, 64573539, 63489079, 63911288, 63937175, 66399802, 64842974, 165887166, 166824805, 166695246, 62626786, 167344702, 167799665, 65480581, 65242908, 65518489, 6406070, 65477380]",organelle part
GO:0044459,0.531,2.390,0.000e+00,2.728e-05,119,29,"[168161151, 28240379, 40674602, 62545344, 21258712, 63132923, 65516796, 66978191, 31357673, 32955361, 40686252, 13203808, 32157942, 13841186, 6347838, 12378604, 8187004, 62946388, 6934942, 47513944, 37969849, 63199719, 10732421, 27603383, 62055617, 33383613, 63129880, 61865720, 63078357]",plasma membrane part
GO:1990351,0.531,2.387,0.000e+00,2.182e-05,117,29,"[168161151, 28240379, 40674602, 62545344, 21258712, 63132923, 65516796, 66978191, 31357673, 32955361, 40686252, 13203808, 32157942, 13841186, 6347838, 12378604, 8187004, 62946388, 6934942, 47513944, 37969849, 63199719, 10732421, 27603383, 62055617, 33383613, 63129880, 61865720, 63078357]",transporter complex
GO:1904949,0.531,2.386,0.000e+00,1.819e-05,117,29,"[168161151, 28240379, 40674602, 62545344, 21258712, 63132923, 65516796, 66978191, 31357673, 32955361, 40686252, 13203808, 32157942, 13841186, 6347838, 12378604, 8187004, 62946388, 6934942, 47513944, 37969849, 63199719, 10732421, 27603383, 62055617, 33383613, 63129880, 61865720, 63078357]",ATPase complex
GO:0098797,0.531,2.386,0.000e+00,1.559e-05,117,29,"[168161151, 28240379, 40674602, 62545344, 21258712, 63132923, 65516796, 66978191, 31357673, 32955361, 40686252, 13203808, 32157942, 13841186, 63

In [31]:
cc_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0000015,-0.870,-3.496,0.000e+00,0.000e+00,43,19,"[62743325, 62658191, 22213675, 18120033, 66797374, 62783504, 3624425, 20853105, 10876818, 45981629, 46324053, 62809084, 21126143, 5189919, 20745054, 66137094, 31709972, 62848403, 17549174]",phosphopyruvate hydratase complex
GO:0044445,-0.870,-3.453,0.000e+00,0.000e+00,47,19,"[62743325, 62658191, 22213675, 18120033, 66797374, 62783504, 3624425, 20853105, 10876818, 45981629, 46324053, 62809084, 21126143, 5189919, 20745054, 66137094, 31709972, 62848403, 17549174]",cytosolic part
GO:1902494,-0.490,-2.699,0.000e+00,0.000e+00,202,56,"[7522635, 22519241, 63652778, 168124324, 168161151, 28240379, 40674602, 62545344, 43045244, 21258712, 65221038, 62743325, 63132923, 65516796, 66978191, 31357673, 32955361, 40686252, 13203808, 32157942, 13841186, 6347838, 12378604, 8187004, 62946388, 6934942, 47513944, 37969849, 63199719, 10732421, 27603383, 62055617, 33383613, 63129880, 61865720, 63078357, 62658191, 63401624, 22213675, 18120033, 66797374, 62783504, 3624425, 20853105, 10876818, 45981629, 46324053, 62809084, 21126143, 5189919, 20745054, 66137094, 31709972, 62848403, 13571215, 17549174]",catalytic complex
GO:0044444,-0.430,-2.607,0.000e+00,0.000e+00,422,80,"[165850968, 11332141, 62054342, 67774188, 4009201, 7522635, 65259195, 22519241, 63652778, 44182220, 167340047, 48590768, 167349682, 168017014, 29341916, 7259645, 67389484, 82054302, 166807349, 74406712, 67061811, 62018747, 166720030, 62017462, 20720618, 13581456, 66350717, 43045244, 65221038, 65886657, 62743325, 66541818, 4774708, 66050469, 62658191, 65480581, 65242908, 65518489, 6406070, 67641346, 66639541, 30291146, 33140401, 7354884, 36091865, 67329767, 67458415, 165823420, 166327905, 68568761, 44131449, 65477380, 22213675, 18120033, 61809381, 66053289, 66797374, 62783504, 65977279, 68748656, 70697066, 21053600, 67469358, 66838239, 56890477, 3624425, 20853105, 10876818, 45981629, 46324053, 62809084, 21126143, 5189919, 20745054, 66137094, 31709972, 62848403, 17549174, 67144272, 66453589]",cytoplasmic part
GO:0005737,-0.471,-2.194,4.234e-04,4.848e-04,264,31,"[63092817, 166239197, 61703706, 62359585, 62702496, 18950035, 54139955, 36306027, 8550259, 62228244, 62468665, 62405280, 62299040, 165706474, 66039359, 1488186, 62492484, 54586926, 64117322, 13567160, 63600850, 48778871, 51629989, 167057179, 64066327, 61714907, 166468321, 62120168, 167938999, 13569694, 63464232]",cytoplasm
GO:0043229,-0.340,-1.842,6.066e-03,9.561e-03,327,51,"[165850968, 11332141, 62054342, 67774188, 4009201, 44182220, 167340047, 48590768, 167349682, 168017014, 29341916, 7259645, 67389484, 82054302, 166807349, 74406712, 67061811, 62018747, 166720030, 62017462, 20720618, 13581456, 66350717, 65886657, 66541818, 4774708, 66050469, 67641346, 66639541, 30291146, 33140401, 7354884, 36091865, 67329767, 67458415, 165823420, 166327905, 68568761, 44131449, 65477380, 66053289, 65977279, 68748656, 70697066, 21053600, 67469358, 61950988, 66838239, 56890477, 67144272, 66453589]",intracellular organelle
GO:0043232,-0.340,-1.838,3.035e-03,8.311e-03,320,51,"[165850968, 11332141, 62054342, 67774188, 4009201, 44182220, 167340047, 48590768, 167349682, 168017014, 29341916, 7259645, 67389484, 82054302, 166807349, 74406712, 67061811, 62018747, 166720030, 62017462, 20720618, 13581456, 66350717, 65886657, 66541818, 4774708, 66050469, 67641346, 66639541, 30291146, 33140401, 7354884, 36091865, 67329767, 67458415, 165823420, 166327905, 68568761, 44131449, 65477380, 66053289, 65977279, 68748656, 70697066, 21053600, 67469358, 61950988, 66838239, 56890477, 67144272, 66453589]",intracellular non-membrane-bounded organelle
GO:0005840,-0.328,-1.759,7.912e-03,1.312e-02,319,50,"[165850968, 11332141, 62054342, 67774188, 4009201, 44182220, 167340047, 48590768, 167349682, 168017014, 29341916, 7259645, 67389484, 82054302, 166807349, 74406712, 67061811, 62018747, 166720030, 62017462, 20720618, 13581456, 66350717, 65886657, 66541818, 47747

## RT vs WT Human/Mouse

In [32]:
out_dir = "RT_WT_hm_gsea"
df = pd.read_csv(os.path.join(BASE,"RT_WT_deseq_results.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_wt = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_wt = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-04-04 12:45:23,893 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-04-04 12:45:23,945 0007 gene_sets used for further statistical testing.....
2017-04-04 12:45:23,945 Start to run GSEA...Might take a while..................
2017-04-04 12:45:24,847 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-04-04 12:45:26,811 Congratulations...GSEAPY run successfully...............
2017-04-04 12:45:26,830 Parsing data files for GSEA.............................
2017-04-04 12:45:26,889 0010 gene_sets used for further statistical testi

In [33]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0030246,0.577,1.538,0.044,0.193,31,7,"[165643482, 165647385, 61499891, 165653125, 165647386, 165643378, 165643381]",carbohydrate binding


In [34]:
mf_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [35]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0065007,0.649,1.997,8.066e-04,0.012,86,11,"[165648111, 165648112, 61499238, 61499613, 61503068, 165636196, 165642139, 165646068, 165661488, 61457133, 165661076]",biological regulation
GO:0050789,0.663,1.845,6.288e-03,0.021,80,8,"[165648111, 165648112, 61499238, 61503068, 165636196, 165642139, 165646068, 61457133]",regulation of biological process


In [36]:
bp_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


## RT vs WT Non-Human/Mouse

In [37]:
out_dir = "RT_WT_gsea"
df = pd.read_csv(os.path.join(BASE,"RT_WT_deseq_results.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[~df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)

In [38]:
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_wt = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_wt = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
cc_rt = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
cc_wt = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-04-04 12:45:31,283 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-04-04 12:45:31,548 0076 gene_sets used for further statistical testing.....
2017-04-04 12:45:31,549 Start to run GSEA...Might take a while..................
2017-04-04 12:46:44,974 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-04-04 12:46:50,503 Congratulations...GSEAPY run successfully...............
2017-04-04 12:46:50,528 Parsing data files for GSEA.............................
2017-04-04 12:46:50,810 0093 gene_sets used for further statistical testi

In [39]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0004618,0.625,2.605,0.000e+00,1.942e-04,78,21,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4559493, 4737261, 45207415, 33420375, 22187812, 67028544, 59725586, 54913980, 4220178, 18128369, 65935692, 168089107, 66846572, 62829929, 63123662]",phosphoglycerate kinase activity
GO:0016774,0.570,2.481,0.000e+00,1.457e-04,111,24,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4559493, 4737261, 45207415, 33420375, 22187812, 67028544, 59725586, 54913980, 3884277, 4220178, 18128369, 63123377, 65935692, 168089107, 66846572, 62829929, 13604374, 63123662]","phosphotransferase activity, carboxyl group as acceptor"
GO:0003723,0.339,2.072,1.527e-04,1.489e-02,367,79,"[7655641, 64669875, 20323736, 33709212, 5730052, 10185112, 8974409, 5141220, 17035162, 8283720, 52960359, 65259195, 26879329, 23226729, 6962424, 15891127, 29630113, 23720124, 8107184, 20132923, 11332141, 165850968, 16968363, 40487328, 38183434, 5365520, 15916898, 61892960, 3492858, 4100571, 62054342, 41621220, 62224782, 10593880, 36697039, 61884939, 38302786, 23721515, 33208200, 30459755, 12493950, 31342205, 12690700, 10631876, 5753625, 30556349, 61893464, 44182220, 68927583, 4582849, 59885854, 167619418, 64742627, 65657914, 62896518, 65777235, 62322368, 9904239, 21252244, 36091865, 67329767, 65477380, 67968100, 37858962, 33732979, 66744947, 20822927, 67292411, 61756207, 166327905, 165823420, 67458415, 65669111, 70697066, 21053600, 66838239, 56890477, 29262834, 64167079]",RNA binding
GO:0022892,0.545,2.058,1.214e-03,1.275e-02,89,16,"[3885199, 1994037, 29978920, 33721585, 38210955, 54503535, 21357752, 32363526, 1809293, 268176, 21432496, 12867643, 27807827, 63810043, 64582513, 165706474]",substrate-specific transporter activity
GO:0022857,0.476,1.959,4.743e-03,2.366e-02,212,20,"[3885199, 1994037, 29978920, 33721585, 38210955, 54503535, 21357752, 32363526, 1809293, 268176, 21432496, 12867643, 47513944, 37969849, 63374013, 27807827, 28273397, 63810043, 64582513, 165706474]",transmembrane transporter activity
GO:0001882,0.356,1.873,4.115e-03,3.969e-02,398,44,"[5888442, 80382425, 5730052, 10185112, 8974409, 21138590, 12661961, 32794085, 26879329, 40640683, 69205977, 12999155, 23375966, 6243639, 4100571, 61982117, 36454371, 61884939, 49556056, 61106614, 5753625, 30556349, 16579503, 61893464, 69572327, 15782696, 80394470, 67719393, 15769577, 168124362, 167619418, 38421939, 66146108, 62727257, 10288858, 13297453, 37822499, 20642101, 43782565, 51285943, 15295447, 165685554, 29262834, 3557492]",nucleoside binding
GO:0016301,0.327,1.845,2.740e-03,4.271e-02,480,57,"[30875488, 17585926, 8107297, 14006785, 4543087, 8170293, 4559493, 4737261, 45207415, 21799656, 33420375, 62247377, 167526299, 17583418, 22187812, 61635847, 67028544, 59725586, 54913980, 3884277, 52517748, 62422222, 63185681, 13585577, 52152229, 66492431, 63266944, 61708790, 4220178, 18128369, 61733256, 67137330, 47809732, 66416823, 59382202, 62986062, 40653126, 42129940, 62942343, 36117032, 51371961, 48698283, 62102911, 63123377, 65935692, 168089107, 62109485, 66846572, 51336959, 64346968, 55261886, 48298182, 62829929, 59827295, 13604374, 63123662, 62098267]",kinase activity
GO:0016614,0.366,1.781,1.184e-02,5.954e-02,150,34,"[10318713, 13684061, 6155443, 7011122, 11147909, 166616289, 16967630, 63958498, 56134344, 26904631, 27290903, 29001612, 16965587, 29983272, 20853009, 7496609, 27407984, 63505555, 12905190, 8550259, 62329349, 62298783, 20839749, 51213068, 51313098, 62853917, 63332989, 167277486, 63235820, 51115316, 63662802, 63648739, 63464232, 61809381]","oxidoreductase activity, acting on CH-OH group of donors"
GO:0003735,0.272,1.748,3.471e-03,6.653e-02,360,95,"[7655641, 64669875, 20323736, 33709212, 19107849, 29630217, 5141220, 17035162, 8283720, 60453073, 5645650, 65259195, 28829095, 165941560, 25902959, 20862748, 23226729, 6962424, 15891127, 29630113, 23720124, 6288540, 8107184, 20132923, 11332141, 165850968, 16968363, 40487328, 

In [40]:
mf_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0004634,-0.577,-2.335,0.000e+00,0.005,43,16,"[29629717, 23089117, 17564385, 17549174, 20853105, 62848403, 31709972, 62566960, 3624425, 20745054, 46324053, 45981629, 5189919, 10876818, 21126143, 62809084]",phosphopyruvate hydratase activity
GO:0016798,-0.427,-2.163,2.532e-04,0.014,76,30,"[11310495, 11409356, 13575738, 11310502, 167568891, 8550179, 53177315, 66536404, 18978935, 167683569, 62488243, 62035953, 63376055, 63922748, 168249536, 167639021, 61777171, 62521472, 36284522, 167252077, 61761442, 166205751, 51133511, 61941182, 167549118, 62146975, 13605039, 13571215, 62450924, 65262333]","hydrolase activity, acting on glycosyl bonds"
GO:0016835,-0.468,-2.115,1.449e-03,0.014,85,21,"[29629717, 23089117, 80889105, 17564385, 62217311, 20611399, 17549174, 20853105, 62848403, 31709972, 62566960, 21791871, 51262100, 3624425, 20745054, 46324053, 45981629, 5189919, 10876818, 21126143, 62809084]",carbon-oxygen lyase activity
GO:0030246,-0.575,-2.084,1.827e-03,0.013,31,12,"[18135548, 11310502, 61777171, 20895349, 63655654, 61768284, 36284522, 61721575, 51133511, 13571510, 167549118, 13571215]",carbohydrate binding
GO:0004553,-0.409,-2.055,1.979e-03,0.013,75,29,"[11310495, 11409356, 13575738, 11310502, 167568891, 8550179, 53177315, 66536404, 18978935, 167683569, 62488243, 62035953, 63376055, 63922748, 168249536, 167639021, 61777171, 62521472, 36284522, 167252077, 61761442, 166205751, 51133511, 61941182, 167549118, 62146975, 13605039, 13571215, 62450924]","hydrolase activity, hydrolyzing O-glycosyl compounds"
GO:0000287,-0.279,-1.534,4.113e-02,0.303,113,39,"[29629717, 9654148, 23089117, 25933508, 17589318, 26904631, 27290903, 29001612, 29983272, 20853009, 7496609, 28314259, 22516735, 8551124, 17564385, 62071220, 17549174, 20853105, 62853917, 60553857, 62848403, 31709972, 62176760, 63235820, 63662802, 63648739, 62566960, 61756207, 13574916, 65218941, 3624425, 28234013, 20745054, 46324053, 45981629, 5189919, 10876818, 21126143, 62809084]",magnesium ion binding
GO:0046872,-0.214,-1.499,3.099e-02,0.307,488,87,"[4715355, 29629717, 10318713, 9654148, 23089117, 6998407, 25933508, 7618450, 167451486, 63441836, 17589318, 4416015, 50252877, 166487064, 26904631, 27290903, 29001612, 29983272, 20853009, 7496609, 62515931, 28314259, 27407984, 62114668, 22516735, 64427981, 42827541, 64182902, 8551124, 17564385, 36356543, 64211477, 39493763, 47787704, 65323654, 66198980, 66019350, 65700587, 64424621, 36510691, 62071220, 22197507, 62073848, 168109762, 64823101, 17549174, 20853105, 62853917, 60553857, 51141976, 166574781, 62848403, 31709972, 62176760, 54721784, 63235820, 167849528, 63662802, 63648739, 22606442, 62960491, 10323127, 167278898, 62566960, 61756207, 13574916, 41253846, 167279252, 14653772, 65218941, 62216988, 65690420, 65762645, 166491001, 64479173, 3624425, 28234013, 61809381, 20745054, 46324053, 45981629, 5189919, 10876818, 21126143, 62809084, 61608244, 166278783]",metal ion binding


In [41]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0040011,0.362,2.061,6.275e-04,0.058,274,57,"[168171488, 167307068, 13580518, 13586931, 167570053, 62303301, 42831732, 64855697, 64780867, 62417891, 62405618, 62136808, 62719918, 62849941, 57751592, 52127537, 13574239, 48437829, 68252636, 62771320, 63144665, 167641590, 62619244, 62923262, 62926584, 166007447, 28438217, 63195134, 63275290, 63842059, 64065097, 66593668, 62002315, 167738934, 66403957, 62565570, 62396394, 11407241, 167337265, 167047560, 13589494, 165880634, 167438835, 67398658, 63489079, 166530730, 168112544, 166818826, 64573539, 168001411, 63911288, 63937175, 168185530, 168106741, 168173199, 166250719, 166636440]",locomotion
GO:0048870,0.362,2.046,0.000e+00,0.034,269,57,"[168171488, 167307068, 13580518, 13586931, 167570053, 62303301, 42831732, 64855697, 64780867, 62417891, 62405618, 62136808, 62719918, 62849941, 57751592, 52127537, 13574239, 48437829, 68252636, 62771320, 63144665, 167641590, 62619244, 62923262, 62926584, 166007447, 28438217, 63195134, 63275290, 63842059, 64065097, 66593668, 62002315, 167738934, 66403957, 62565570, 62396394, 11407241, 167337265, 167047560, 13589494, 165880634, 167438835, 67398658, 63489079, 166530730, 168112544, 166818826, 64573539, 168001411, 63911288, 63937175, 168185530, 168106741, 168173199, 166250719, 166636440]",cell motility
GO:0006928,0.362,2.043,9.539e-04,0.023,270,57,"[168171488, 167307068, 13580518, 13586931, 167570053, 62303301, 42831732, 64855697, 64780867, 62417891, 62405618, 62136808, 62719918, 62849941, 57751592, 52127537, 13574239, 48437829, 68252636, 62771320, 63144665, 167641590, 62619244, 62923262, 62926584, 166007447, 28438217, 63195134, 63275290, 63842059, 64065097, 66593668, 62002315, 167738934, 66403957, 62565570, 62396394, 11407241, 167337265, 167047560, 13589494, 165880634, 167438835, 67398658, 63489079, 166530730, 168112544, 166818826, 64573539, 168001411, 63911288, 63937175, 168185530, 168106741, 168173199, 166250719, 166636440]",movement of cell or subcellular component
GO:0072350,0.599,1.836,1.027e-02,0.100,29,9,"[13684061, 78477008, 80889105, 26904631, 29983272, 20853009, 7496609, 3740901, 62853917]",tricarboxylic acid metabolic process
GO:0044267,0.280,1.817,3.021e-03,0.093,413,101,"[7655641, 64669875, 20323736, 33709212, 27212193, 38310093, 19107849, 29630217, 5141220, 17035162, 8283720, 60453073, 5645650, 65259195, 28829095, 165941560, 25902959, 20862748, 23226729, 6962424, 15891127, 29630113, 23720124, 6288540, 8107184, 20132923, 11332141, 165850968, 16968363, 40487328, 38183434, 5365520, 15916898, 3492858, 62054342, 41621220, 62224782, 23449836, 10593880, 36697039, 38302786, 23721515, 33208200, 30459755, 12493950, 31342205, 12690700, 21157792, 44182220, 3576806, 68927583, 4582849, 59885854, 64742627, 65657914, 28150928, 69613400, 38731273, 67911420, 39007615, 66583625, 65777235, 66782633, 66339926, 9904239, 21252244, 65506212, 65977068, 167623679, 64474525, 42272589, 167278898, 36091865, 66752541, 67329767, 65477380, 167990362, 65626218, 67968100, 67247141, 37858962, 33732979, 66744947, 20822927, 38249992, 67292411, 66053289, 13581456, 67383571, 166327905, 165823420, 67458415, 68373997, 10276368, 65669111, 70697066, 21053600, 66838239, 56890477, 33140401, ...]",cellular protein metabolic process
GO:1902578,0.457,1.815,8.095e-03,0.078,132,18,"[3885199, 1994037, 29978920, 33721585, 38210955, 54503535, 21357752, 32363526, 1809293, 268176, 21432496, 29629513, 12867643, 27807827, 63810043, 64582513, 165706474, 67703402]",single-organism localization
GO:0006412,0.269,1.737,6.115e-03,0.118,372,96,"[7655641, 64669875, 20323736, 33709212, 19107849, 29630217, 5141220, 17035162, 8283720, 60453073, 5645650, 65259195, 28829095, 165941560, 25902959, 20862748, 23226729, 6962424, 15891127, 29630113, 23720124, 6288540, 8107184, 20132923, 11332141, 165850968, 16968363, 40487328, 38183434, 5365520, 15916898, 3492858, 62054342, 41621220, 62224782, 23449836, 10593880, 36697039, 38302786, 23721515,

In [42]:
bp_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0009081,-0.577,-1.936,0.006,0.131,47,10,"[20839749, 51213068, 63289385, 51313098, 13442927, 165749819, 63235820, 63662802, 63648739, 63600850]",branched-chain amino acid metabolic process
GO:0009082,-0.571,-1.659,0.035,0.365,41,7,"[20839749, 51213068, 51313098, 63235820, 63662802, 63648739, 63600850]",branched-chain amino acid biosynthetic process
GO:0050794,-0.610,-1.657,0.032,0.246,36,6,"[165873031, 167750483, 59827295, 66251821, 37727049, 65993244]",regulation of cellular process
GO:0019222,-0.612,-1.644,0.039,0.199,59,6,"[61892960, 10631876, 59827295, 66251821, 37727049, 65993244]",regulation of metabolic process


In [43]:
cc_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0044446,0.493,2.191,6.741e-04,0.005,169,25,"[7655641, 64669875, 20323736, 33709212, 17035162, 8283720, 65259195, 23226729, 6962424, 15891127, 6288540, 20132923, 3492858, 41621220, 10593880, 44182220, 68927583, 59885854, 64742627, 65657914, 9904239, 21252244, 65477380, 38249992, 65669111]",intracellular organelle part
GO:0015934,0.642,2.188,7.048e-04,0.003,63,12,"[7655641, 64669875, 20323736, 33709212, 17035162, 8283720, 15891127, 41621220, 44182220, 59885854, 64742627, 38249992]",large ribosomal subunit
GO:0044391,0.493,2.172,6.767e-04,0.002,159,25,"[7655641, 64669875, 20323736, 33709212, 17035162, 8283720, 65259195, 23226729, 6962424, 15891127, 6288540, 20132923, 3492858, 41621220, 10593880, 44182220, 68927583, 59885854, 64742627, 65657914, 9904239, 21252244, 65477380, 38249992, 65669111]",ribosomal subunit
GO:0042995,0.362,2.062,1.571e-04,0.004,269,57,"[168171488, 167307068, 13580518, 13586931, 167570053, 62303301, 42831732, 64855697, 64780867, 62417891, 62405618, 62136808, 62719918, 62849941, 57751592, 52127537, 13574239, 48437829, 68252636, 62771320, 63144665, 167641590, 62619244, 62923262, 62926584, 166007447, 28438217, 63195134, 63275290, 63842059, 64065097, 66593668, 62002315, 167738934, 66403957, 62565570, 62396394, 11407241, 167337265, 167047560, 13589494, 165880634, 167438835, 67398658, 63489079, 166530730, 168112544, 166818826, 64573539, 168001411, 63911288, 63937175, 168185530, 168106741, 168173199, 166250719, 166636440]",cell projection
GO:0044422,0.345,2.038,7.831e-04,0.004,402,67,"[7655641, 64669875, 20323736, 33709212, 17035162, 8283720, 65259195, 23226729, 6962424, 15891127, 6288540, 20132923, 3492858, 168171488, 41621220, 167307068, 10593880, 13580518, 13586931, 167570053, 62303301, 42831732, 64855697, 64780867, 62417891, 62405618, 62136808, 62719918, 62849941, 13574239, 62771320, 63144665, 62619244, 62923262, 62926584, 28438217, 44182220, 62002315, 68927583, 167738934, 66403957, 11407241, 167337265, 167047560, 13589494, 59885854, 64742627, 167438835, 67398658, 63489079, 166530730, 168112544, 166818826, 64573539, 168001411, 63911288, 63937175, 65657914, 168185530, 168106741, 9904239, 21252244, 65477380, 38249992, 65669111, 168173199, 166250719]",organelle part
GO:0044463,0.350,1.816,6.970e-03,0.019,229,42,"[168171488, 167307068, 13580518, 13586931, 167570053, 62303301, 42831732, 64855697, 64780867, 62417891, 62405618, 62136808, 62719918, 62849941, 13574239, 62771320, 63144665, 62619244, 62923262, 62926584, 28438217, 62002315, 167738934, 66403957, 11407241, 167337265, 167047560, 13589494, 167438835, 67398658, 63489079, 166530730, 168112544, 166818826, 64573539, 168001411, 63911288, 63937175, 168185530, 168106741, 168173199, 166250719]",cell projection part
GO:0005622,0.312,1.811,4.740e-03,0.018,306,63,"[7655641, 64669875, 20323736, 33709212, 5730052, 10185112, 19107849, 29630217, 8974409, 5141220, 60453073, 5645650, 65259195, 28829095, 165941560, 26879329, 25902959, 20862748, 15891127, 4100571, 41621220, 23449836, 3884277, 61884939, 5753625, 30556349, 61893464, 44182220, 3576806, 59885854, 167619418, 64742627, 65657914, 63123377, 69613400, 38731273, 21252244, 64474525, 42272589, 36091865, 67329767, 65477380, 67968100, 37858962, 33732979, 66744947, 20822927, 38249992, 67292411, 67383571, 166327905, 165823420, 67458415, 68373997, 10276368, 65669111, 70697066, 21053600, 66838239, 56890477, 33140401, 13604374, 29262834]",intracellular
GO:1990904,0.272,1.742,4.833e-03,0.025,360,95,"[7655641, 64669875, 20323736, 33709212, 19107849, 29630217, 5141220, 17035162, 8283720, 60453073, 5645650, 65259195, 28829095, 165941560, 25902959, 20862748, 23226729, 6962424, 15891127, 29630113, 23720124, 6288540, 8107184, 20132923, 11332141, 165850968, 16968363, 40487328, 38183434, 5365520, 15916898, 3492858, 62054342, 41621220, 62224782, 23449836, 10593880, 36697039, 38302786, 23721515, 33208200, 30459755, 12493950, 31342205, 12690700, 44182220, 3576806, 68927583,

In [44]:
cc_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0000015,-0.577,-2.341,0.000e+00,5.354e-04,43,16,"[29629717, 23089117, 17564385, 17549174, 20853105, 62848403, 31709972, 62566960, 3624425, 20745054, 46324053, 45981629, 5189919, 10876818, 21126143, 62809084]",phosphopyruvate hydratase complex
GO:0044445,-0.565,-2.333,2.360e-04,3.407e-04,47,17,"[29629717, 23089117, 17564385, 63672434, 17549174, 20853105, 62848403, 31709972, 62566960, 3624425, 20745054, 46324053, 45981629, 5189919, 10876818, 21126143, 62809084]",cytosolic part
GO:1902494,-0.468,-2.254,2.496e-04,4.542e-04,202,26,"[29629717, 23089117, 11310502, 22519241, 43045244, 65221038, 17564385, 47513944, 37969849, 63374013, 17549174, 20853105, 28273397, 62848403, 31709972, 63401624, 62566960, 3624425, 20745054, 46324053, 45981629, 5189919, 10876818, 21126143, 62809084, 13571215]",catalytic complex
GO:0016020,-0.391,-1.604,4.121e-02,3.126e-02,75,17,"[21357752, 21687303, 62114668, 64265900, 46871761, 63823493, 167156307, 61699598, 166216926, 62830069, 27635679, 64013758, 166593522, 61660751, 65386242, 167150261, 165685567]",membrane


## RT vs Rag Human/Mouse

In [45]:
out_dir = "RT_Rag_hm_gsea"
df = pd.read_csv(os.path.join(BASE,"RT_Rag_deseq_results.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_rag = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_rag = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-04-04 12:48:56,283 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-04-04 12:48:56,334 0008 gene_sets used for further statistical testing.....
2017-04-04 12:48:56,335 Start to run GSEA...Might take a while..................
2017-04-04 12:48:57,418 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-04-04 12:48:59,534 Congratulations...GSEAPY run successfully...............
2017-04-04 12:48:59,552 Parsing data files for GSEA.............................
2017-04-04 12:48:59,606 0013 gene_sets used for further statistical testi

In [46]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0008270,0.654,1.551,0.034,0.257,89,6,"[165648111, 165648112, 61533297, 165637224, 61507483, 61501312]",zinc ion binding


In [47]:
mf_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [48]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0065007,0.612,1.788,0.003,0.051,86,13,"[165648111, 165648112, 61499238, 61499613, 61503068, 165636196, 165642139, 165646068, 165661488, 61457133, 61499612, 165666724, 165661076]",biological regulation
GO:0050789,0.653,1.741,0.007,0.042,80,9,"[165648111, 165648112, 61499238, 61503068, 165636196, 165642139, 165646068, 61457133, 165666724]",regulation of biological process


In [49]:
bp_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


## RT vs Rag Non-Human/Mouse

In [50]:
out_dir = "RT_Rag_gsea"
df = pd.read_csv(os.path.join(BASE,"RT_Rag_deseq_results.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[~df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)

In [51]:
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_rag = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_rag = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
cc_rt = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
cc_rag = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-04-04 12:49:05,166 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-04-04 12:49:05,444 0077 gene_sets used for further statistical testing.....
2017-04-04 12:49:05,445 Start to run GSEA...Might take a while..................
2017-04-04 12:50:49,431 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-04-04 12:50:55,293 Congratulations...GSEAPY run successfully...............
2017-04-04 12:50:55,313 Parsing data files for GSEA.............................
2017-04-04 12:50:55,626 0090 gene_sets used for further statistical testi

In [52]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0016781,0.530,3.223,0.000e+00,0.000e+00,302,195,"[61708790, 61884808, 36565362, 61724660, 51077372, 52166470, 61735080, 61731944, 10439591, 10550160, 36213075, 20333280, 7539645, 10899544, 11983619, 1654966, 7971161, 2550329, 2717161, 3723795, 64511620, 65325384, 6852416, 17374966, 33165099, 7297234, 39495835, 42033080, 45464484, 40105724, 61736606, 45577963, 67215245, 66119965, 12153742, 61721447, 42801234, 42222587, 166305458, 45352916, 31262207, 55837820, 36616158, 61723254, 14693091, 61792464, 7200027, 16289890, 7114893, 61686338, 21361007, 61722089, 51148156, 61731522, 61736645, 33383509, 6343409, 15304368, 9050982, 8071617, 6702801, 21125442, 20673079, 7309750, 7477105, 52796995, 21786113, 17395074, 6186763, 4228803, 3545035, 61725747, 6637923, 21265599, 8033819, 22855681, 26027080, 18675504, 10235090, 10296556, 10630096, 11072872, 29952533, 15038592, 15339324, 15551744, 32548173, 15894944, 32633754, 16859845, 30581852, 18127587, 31357883, 8541815, 3902762, 8042238, 2749207, 4246480, 4841345, 4854118, ...]","phosphotransferase activity, paired acceptors"
GO:0003723,0.663,3.209,0.000e+00,0.000e+00,367,55,"[7655641, 64669875, 20323736, 33709212, 5141220, 8283720, 17035162, 52960359, 10185112, 8974409, 23226729, 6962424, 5730052, 15891127, 29630113, 23720124, 8107184, 20132923, 40487328, 16968363, 38183434, 15916898, 5365520, 61892960, 4100571, 3492858, 26879329, 41621220, 62224782, 10593880, 36697039, 61884939, 23721515, 38302786, 33208200, 12493950, 30459755, 31342205, 12690700, 10631876, 67469358, 66402327, 68927583, 5753625, 30556349, 61780119, 65259195, 21408386, 167619418, 65777235, 62322368, 166720030, 10899352, 29262834, 65669111]",RNA binding
GO:0097747,0.621,3.010,0.000e+00,0.000e+00,345,55,"[12999155, 23375966, 6243639, 16965124, 18068412, 61596152, 61782419, 4493173, 9867739, 21360327, 61595514, 74304202, 61719675, 48895505, 52385306, 6752061, 17299375, 62892851, 63204893, 8860974, 17354404, 8187869, 6675473, 6211757, 40202302, 10193140, 62063123, 52349047, 52155195, 64978053, 62155980, 61696438, 61687871, 41455330, 61761062, 41165392, 62823286, 63261481, 62902575, 66762307, 62874585, 62315770, 37387955, 51691431, 166882954, 43146673, 61079203, 21252862, 21868198, 7691951, 60772420, 56745534, 50065172, 44186767, 16841057]",RNA polymerase activity
GO:0003677,0.583,2.844,0.000e+00,0.000e+00,369,58,"[12999155, 23375966, 6243639, 16965124, 18068412, 61596152, 61782419, 4493173, 9867739, 21360327, 61595514, 74304202, 61719675, 48895505, 52385306, 6752061, 17299375, 62892851, 63204893, 8860974, 17354404, 8187869, 6675473, 6211757, 40202302, 10193140, 62063123, 52349047, 52155195, 64978053, 62155980, 61696438, 61687871, 41455330, 61761062, 41165392, 62823286, 63261481, 62902575, 66762307, 62874585, 62315770, 37387955, 51691431, 166882954, 43146673, 61079203, 21252862, 21868198, 7691951, 60772420, 56745534, 50065172, 44186767, 62437043, 46260937, 51907206, 16841057]",DNA binding
GO:0016779,0.559,2.839,0.000e+00,0.000e+00,463,67,"[12999155, 23375966, 6243639, 18275642, 61892960, 16965124, 10631876, 18068412, 61596152, 61782419, 4493173, 9867739, 21360327, 61595514, 74304202, 61719675, 48895505, 52385306, 6752061, 17299375, 62892851, 63204893, 8860974, 17354404, 8187869, 6675473, 6211757, 40202302, 10193140, 62063123, 52349047, 52155195, 64978053, 62155980, 61696438, 61687871, 41455330, 61761062, 41165392, 62823286, 63261481, 62902575, 66762307, 62874585, 62315770, 37387955, 51691431, 166882954, 43146673, 61780119, 61079203, 21252862, 21868198, 7691951, 60772420, 56745534, 50065172, 44186767, 13570629, 62718684, 59156012, 63039326, 63112646, 65657046, 57028376, 40551040, 16841057]",nucleotidyltransferase activity
GO:0016301,0.433,2.667,0.000e+00,0.000e+00,480,207,"[30875488, 17585926, 8107297, 14006785, 4543087, 4559493, 8170293, 4737261, 45207415, 33420375, 22187812, 61635847, 67028544, 61499980, 14397111, 59725586, 54913980, 3884277, 17583418, 617087

In [53]:
mf_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0009055,-0.646,-2.467,0.000,4.325e-04,38,18,"[63507869, 63623628, 64864375, 63379850, 64572991, 62942791, 64649656, 49911241, 45703293, 61957888, 21346720, 63079626, 6263025, 63389326, 21834593, 64646230, 63098496, 167701862]",electron carrier activity
GO:0051540,-0.597,-2.403,0.000,3.892e-04,149,21,"[80889105, 61600538, 61963309, 61598826, 64109059, 61600015, 58687019, 62139539, 48340027, 61601993, 168109762, 57602627, 61621955, 61957888, 21346720, 62751142, 167701862, 167744672, 63901915, 61608244, 166278783]",metal cluster binding
GO:0016887,-0.411,-2.327,0.000,5.190e-04,249,73,"[1994037, 32363526, 1809293, 268176, 13568664, 63078357, 63959850, 6924770, 63452797, 77351967, 43878788, 67306374, 13441580, 10689069, 66551099, 47459480, 56069133, 63684566, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 5247630, 6347838, 14468787, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 65627086, 63468226, 9085008, 63133221, 20629788, 64420003, 67347833, 63199719, 63281680, 12378604, 40686252, 62354250, 168161151, 66854840, 40674602, 28240379, 63132923, 62545344, 167915219, 45006276, 28273397, 29219301, 62344870, 11992086, 10265600, 17618404, 56076521, 66939789, 13413281, 29505496, 9507913, 22209574, 31262467, 33706145, 63301199, 13842682, 63695501, 63374013]",ATPase activity
GO:0016820,-0.366,-1.940,0.001,2.104e-02,190,56,"[1994037, 32363526, 1809293, 268176, 13568664, 63078357, 6924770, 63452797, 77351967, 10689069, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 6347838, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 65627086, 63468226, 63133221, 20629788, 63199719, 12378604, 40686252, 62354250, 168161151, 65516796, 40674602, 28240379, 66978191, 63132923, 62545344, 167915219, 165851092, 28273397, 47513944, 62344870, 11992086, 10265600, 17618404, 56076521, 13413281, 29505496, 22209574, 31262467, 33706145, 37969849, 63374013]","hydrolase activity, acting on acid anhydrides, catalyzing transmembrane movement of substances"
GO:1901681,-0.574,-1.801,0.012,4.704e-02,80,10,"[3740901, 61600538, 61598826, 61600015, 61601993, 168109762, 57602627, 61621955, 61608244, 166278783]",sulfur compound binding
GO:0030976,-0.574,-1.790,0.015,4.238e-02,79,10,"[3740901, 61600538, 61598826, 61600015, 61601993, 168109762, 57602627, 61621955, 61608244, 166278783]",thiamine pyrophosphate binding
GO:0005215,-0.305,-1.790,0.001,3.641e-02,256,83,"[3885199, 1994037, 29978920, 33721585, 38210955, 54503535, 21357752, 32363526, 1809293, 268176, 21432496, 13568664, 63078357, 6924770, 63452797, 77351967, 10689069, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 6347838, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 65627086, 63468226, 63133221, 20629788, 63199719, 12378604, 40686252, 62354250, 168161151, 13571500, 65516796, 40674602, 28240379, 66978191, 63132923, 62545344, 167915219, 165851092, 28273397, 47513944, 167285315, 13587543, 166593522, 167348034, 166208277, 27807827, 167156307, 62344870, 11992086, 10265600, 17618404, 56076521, 13570595, 13413281, 29505496, 22209574, 31262467, 33706145, 37969849, 63374013, 28423332, 166294804, 29374081, 165685567, 11412988, 167645582, 165706474, 62660497, 167399046, 165992613, 167279298]",transporter activity
GO:0022804,-0.321,-1.755,0.006,4.083e-02,205,61,"[1994037, 33721585, 21357752, 32363526, 1809293, 268176, 13568664, 63078357, 6924770, 63452797, 77351967, 10689069, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 6347838, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 65627086, 63468226, 63133221, 20629788, 63199719, 12378604, 40686252, 62354250, 168161151, 65516796, 40674602, 28240379, 66978191, 63132923, 62545344, 167915219, 165851092, 28273397, 47513944, 27807827, 62344870, 11992086, 10265600, 17618404, 56076521, 13413281, 29505496, 22209574, 31262467, 33706145, 37969849, 

In [54]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0032774,0.621,2.987,0.000e+00,0.000e+00,345,55,"[12999155, 23375966, 6243639, 16965124, 18068412, 61596152, 61782419, 4493173, 9867739, 21360327, 61595514, 74304202, 61719675, 48895505, 52385306, 6752061, 17299375, 62892851, 63204893, 8860974, 17354404, 8187869, 6675473, 6211757, 40202302, 10193140, 62063123, 52349047, 52155195, 64978053, 62155980, 61696438, 61687871, 41455330, 61761062, 41165392, 62823286, 63261481, 62902575, 66762307, 62874585, 62315770, 37387955, 51691431, 166882954, 43146673, 61079203, 21252862, 21868198, 7691951, 60772420, 56745534, 50065172, 44186767, 16841057]",RNA biosynthetic process
GO:0046034,0.509,2.566,0.000e+00,0.000e+00,379,66,"[30875488, 17585926, 8107297, 14006785, 4543087, 4559493, 8170293, 4737261, 45207415, 29629717, 33420375, 1994037, 9654148, 23089117, 25933508, 32363526, 22187812, 1809293, 268176, 63441836, 67028544, 59725586, 54913980, 30623941, 839701, 12964871, 66137094, 5960044, 17046519, 17549174, 8200910, 65033516, 23090099, 13568664, 30588025, 66797374, 62848403, 31709972, 20853105, 18120033, 22213675, 62887295, 62783504, 3646533, 64427981, 4220178, 63649356, 62446788, 63517822, 65627086, 62748898, 62354250, 166499573, 62754658, 166165052, 63157973, 64380885, 65935692, 168089107, 62344870, 62566960, 165866251, 64479173, 62098267, 64978197, 63123662]",ATP metabolic process
GO:0009141,0.509,2.565,0.000e+00,0.000e+00,381,66,"[30875488, 17585926, 8107297, 14006785, 4543087, 4559493, 8170293, 4737261, 45207415, 29629717, 33420375, 1994037, 9654148, 23089117, 25933508, 32363526, 22187812, 1809293, 268176, 63441836, 67028544, 59725586, 54913980, 30623941, 839701, 12964871, 66137094, 5960044, 17046519, 17549174, 8200910, 65033516, 23090099, 13568664, 30588025, 66797374, 62848403, 31709972, 20853105, 18120033, 22213675, 62887295, 62783504, 3646533, 64427981, 4220178, 63649356, 62446788, 63517822, 65627086, 62748898, 62354250, 166499573, 62754658, 166165052, 63157973, 64380885, 65935692, 168089107, 62344870, 62566960, 165866251, 64479173, 62098267, 64978197, 63123662]",nucleoside triphosphate metabolic process
GO:0009123,0.496,2.520,0.000e+00,0.000e+00,412,67,"[30875488, 17585926, 8107297, 14006785, 4543087, 4559493, 8170293, 4737261, 45207415, 29629717, 33420375, 1994037, 9654148, 23089117, 25933508, 32363526, 22187812, 1809293, 268176, 63441836, 67028544, 59725586, 54913980, 30623941, 839701, 12964871, 66137094, 5960044, 17046519, 17549174, 8200910, 65033516, 23090099, 13568664, 30588025, 66797374, 62848403, 31709972, 20853105, 18120033, 22213675, 62887295, 62783504, 3646533, 64427981, 4220178, 63649356, 62446788, 63517822, 65627086, 62748898, 62354250, 166499573, 62754658, 166165052, 63157973, 64380885, 65935692, 168089107, 62344870, 62566960, 165866251, 63180877, 64479173, 62098267, 64978197, 63123662]",nucleoside monophosphate metabolic process
GO:0009126,0.496,2.502,0.000e+00,2.613e-05,409,67,"[30875488, 17585926, 8107297, 14006785, 4543087, 4559493, 8170293, 4737261, 45207415, 29629717, 33420375, 1994037, 9654148, 23089117, 25933508, 32363526, 22187812, 1809293, 268176, 63441836, 67028544, 59725586, 54913980, 30623941, 839701, 12964871, 66137094, 5960044, 17046519, 17549174, 8200910, 65033516, 23090099, 13568664, 30588025, 66797374, 62848403, 31709972, 20853105, 18120033, 22213675, 62887295, 62783504, 3646533, 64427981, 4220178, 63649356, 62446788, 63517822, 65627086, 62748898, 62354250, 166499573, 62754658, 166165052, 63157973, 64380885, 65935692, 168089107, 62344870, 62566960, 165866251, 63180877, 64479173, 62098267, 64978197, 63123662]",purine nucleoside monophosphate metabolic process
GO:0009132,0.504,2.469,0.000e+00,4.355e-05,304,58,"[30875488, 17585926, 8107297, 14006785, 4543087, 4559493, 8170293, 4737261, 45207415, 29629717, 33420375, 9654148, 23089117, 25933508, 22187812, 63441836, 67028544, 59725586, 54913980, 30623941, 839701, 12964871, 66137094, 5960044, 17046519, 17549174, 8200910, 65033516, 23090099,

In [55]:
bp_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0006790,-0.670,-1.993,0.003,0.033,92,9,"[63603065, 45843396, 61901179, 63849179, 65619550, 62479129, 61880488, 42221176, 20628827]",sulfur compound metabolic process
GO:0022900,-0.617,-1.835,0.010,0.066,72,9,"[61600538, 62232545, 61598826, 61600015, 61601993, 168109762, 61621955, 61608244, 166278783]",electron transport chain
GO:0051234,-0.298,-1.762,0.003,0.075,331,88,"[3885199, 1994037, 29978920, 33721585, 38210955, 54503535, 21357752, 32363526, 1809293, 268176, 21432496, 4416619, 29629513, 13568664, 63078357, 6924770, 63452797, 77351967, 10689069, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 6347838, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 65627086, 63468226, 63133221, 20629788, 63199719, 12378604, 40686252, 62354250, 168161151, 13571500, 65516796, 40674602, 28240379, 66978191, 63132923, 62545344, 167915219, 165851092, 28273397, 47513944, 63780068, 167285315, 13587543, 166593522, 167348034, 166208277, 27807827, 167156307, 62344870, 11992086, 10265600, 17618404, 56076521, 13570595, 13413281, 29505496, 22209574, 31262467, 33706145, 37969849, 63374013, 28423332, 166294804, 29374081, 165685567, 11412988, 62156769, 167645582, 165706474, 62660497, 167399046, 165992613, 167279298, 67703402]",establishment of localization
GO:0051179,-0.298,-1.759,0.001,0.057,332,88,"[3885199, 1994037, 29978920, 33721585, 38210955, 54503535, 21357752, 32363526, 1809293, 268176, 21432496, 4416619, 29629513, 13568664, 63078357, 6924770, 63452797, 77351967, 10689069, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 6347838, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 65627086, 63468226, 63133221, 20629788, 63199719, 12378604, 40686252, 62354250, 168161151, 13571500, 65516796, 40674602, 28240379, 66978191, 63132923, 62545344, 167915219, 165851092, 28273397, 47513944, 63780068, 167285315, 13587543, 166593522, 167348034, 166208277, 27807827, 167156307, 62344870, 11992086, 10265600, 17618404, 56076521, 13570595, 13413281, 29505496, 22209574, 31262467, 33706145, 37969849, 63374013, 28423332, 166294804, 29374081, 165685567, 11412988, 62156769, 167645582, 165706474, 62660497, 167399046, 165992613, 167279298, 67703402]",localization
GO:0040011,-0.398,-1.679,0.020,0.077,274,24,"[167307068, 13580518, 21419243, 48844574, 168130942, 166734854, 62626786, 11413856, 13586933, 22310619, 168062228, 167832826, 167438835, 44091081, 165775316, 166250719, 168173199, 165990924, 63837786, 168185530, 168106741, 167953213, 168192733, 167966912]",locomotion


In [56]:
cc_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0044444,0.443,2.446,0.000e+00,0.000e+00,422,106,"[7655641, 64669875, 20323736, 33709212, 66453589, 67144272, 29629717, 19107849, 29630217, 5141220, 8283720, 17035162, 23089117, 60453073, 5645650, 28829095, 165941560, 3576806, 25902959, 20862748, 23226729, 6962424, 15891127, 29630113, 23720124, 6288540, 8107184, 20132923, 40487328, 16968363, 38183434, 15916898, 5365520, 80889105, 3492858, 41621220, 62224782, 10593880, 23449836, 36697039, 66137094, 23721515, 38302786, 33208200, 12493950, 30459755, 31342205, 17549174, 12690700, 67469358, 3497381, 6962421, 23090099, 65977279, 66402327, 66797374, 62848403, 31709972, 20853105, 18120033, 22213675, 68927583, 62887295, 62783504, 65259195, 21408386, 65777235, 20720618, 66350717, 66050469, 65886657, 66541818, 67061811, 168017014, 166807349, 7259645, 29341916, 67389484, 74406712, 82054302, 48590768, 167340047, 4774708, 166720030, 67911420, 64474525, 39007615, 42272589, 167990362, 66752541, 10899352, 167349682, 67383571, 65506212, 65977068, 65626218, 62566960, 167623679, 38249992, 67911893, ...]",cytoplasmic part
GO:1990904,0.446,2.397,0.000e+00,0.000e+00,360,91,"[7655641, 64669875, 20323736, 33709212, 66453589, 67144272, 19107849, 29630217, 5141220, 8283720, 17035162, 60453073, 5645650, 28829095, 165941560, 3576806, 25902959, 20862748, 23226729, 6962424, 15891127, 29630113, 23720124, 6288540, 8107184, 20132923, 40487328, 16968363, 38183434, 15916898, 5365520, 3492858, 41621220, 62224782, 10593880, 23449836, 36697039, 23721515, 38302786, 33208200, 12493950, 30459755, 31342205, 12690700, 67469358, 3497381, 6962421, 65977279, 66402327, 68927583, 65259195, 21408386, 65777235, 20720618, 66350717, 66050469, 65886657, 66541818, 67061811, 168017014, 166807349, 7259645, 29341916, 67389484, 74406712, 82054302, 48590768, 167340047, 4774708, 166720030, 67911420, 64474525, 39007615, 42272589, 167990362, 66752541, 10899352, 167349682, 67383571, 65506212, 65977068, 65626218, 167623679, 38249992, 67911893, 67774188, 4009201, 166036375, 65669111, 10276368, 13581456]",ribonucleoprotein complex
GO:0005840,0.427,2.263,0.000e+00,1.113e-04,319,82,"[7655641, 64669875, 20323736, 33709212, 66453589, 67144272, 19107849, 29630217, 5141220, 60453073, 5645650, 28829095, 165941560, 3576806, 25902959, 20862748, 15891127, 29630113, 23720124, 8107184, 40487328, 16968363, 38183434, 15916898, 5365520, 3492858, 41621220, 62224782, 10593880, 23449836, 36697039, 23721515, 38302786, 33208200, 12493950, 30459755, 31342205, 12690700, 67469358, 3497381, 6962421, 65977279, 68927583, 21408386, 65777235, 20720618, 66350717, 66050469, 65886657, 66541818, 67061811, 168017014, 166807349, 7259645, 29341916, 67389484, 74406712, 82054302, 48590768, 167340047, 4774708, 166720030, 67911420, 64474525, 39007615, 42272589, 167990362, 66752541, 10899352, 167349682, 67383571, 65506212, 65977068, 65626218, 167623679, 38249992, 67911893, 67774188, 4009201, 166036375, 10276368, 13581456]",ribosome
GO:0043229,0.427,2.256,0.000e+00,1.043e-04,327,82,"[7655641, 64669875, 20323736, 33709212, 66453589, 67144272, 19107849, 29630217, 5141220, 60453073, 5645650, 28829095, 165941560, 3576806, 25902959, 20862748, 15891127, 29630113, 23720124, 8107184, 40487328, 16968363, 38183434, 15916898, 5365520, 3492858, 41621220, 62224782, 10593880, 23449836, 36697039, 23721515, 38302786, 33208200, 12493950, 30459755, 31342205, 12690700, 67469358, 3497381, 6962421, 65977279, 68927583, 21408386, 65777235, 20720618, 66350717, 66050469, 65886657, 66541818, 67061811, 168017014, 166807349, 7259645, 29341916, 67389484, 74406712, 82054302, 48590768, 167340047, 4774708, 166720030, 67911420, 64474525, 39007615, 42272589, 167990362, 66752541, 10899352, 167349682, 67383571, 65506212, 65977068, 65626218, 167623679, 38249992, 67911893, 67774188, 4009201, 166036375, 10276368, 13581456]",intracellular organelle
GO:0043232,0.427,2.250,0.000e+00,1.002e-04,320,82,"[7655641, 64669875, 20323736, 33709212, 66453589, 67144

In [57]:
cc_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0043190,-0.570,-2.912,0.000e+00,0.000,115,47,"[63078357, 6924770, 63452797, 77351967, 10689069, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 6347838, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 63468226, 63133221, 20629788, 63199719, 12378604, 40686252, 168161151, 65516796, 40674602, 28240379, 66978191, 63132923, 62545344, 167915219, 28273397, 47513944, 11992086, 10265600, 17618404, 56076521, 13413281, 29505496, 22209574, 31262467, 33706145, 37969849, 63374013]",ATP-binding cassette (ABC) transporter complex
GO:1990351,-0.570,-2.910,0.000e+00,0.000,117,47,"[63078357, 6924770, 63452797, 77351967, 10689069, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 6347838, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 63468226, 63133221, 20629788, 63199719, 12378604, 40686252, 168161151, 65516796, 40674602, 28240379, 66978191, 63132923, 62545344, 167915219, 28273397, 47513944, 11992086, 10265600, 17618404, 56076521, 13413281, 29505496, 22209574, 31262467, 33706145, 37969849, 63374013]",transporter complex
GO:0098797,-0.570,-2.900,0.000e+00,0.000,117,47,"[63078357, 6924770, 63452797, 77351967, 10689069, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 6347838, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 63468226, 63133221, 20629788, 63199719, 12378604, 40686252, 168161151, 65516796, 40674602, 28240379, 66978191, 63132923, 62545344, 167915219, 28273397, 47513944, 11992086, 10265600, 17618404, 56076521, 13413281, 29505496, 22209574, 31262467, 33706145, 37969849, 63374013]",plasma membrane protein complex
GO:1904949,-0.570,-2.891,0.000e+00,0.000,117,47,"[63078357, 6924770, 63452797, 77351967, 10689069, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 6347838, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 63468226, 63133221, 20629788, 63199719, 12378604, 40686252, 168161151, 65516796, 40674602, 28240379, 66978191, 63132923, 62545344, 167915219, 28273397, 47513944, 11992086, 10265600, 17618404, 56076521, 13413281, 29505496, 22209574, 31262467, 33706145, 37969849, 63374013]",ATPase complex
GO:0044459,-0.570,-2.890,0.000e+00,0.000,119,47,"[63078357, 6924770, 63452797, 77351967, 10689069, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 6347838, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 63468226, 63133221, 20629788, 63199719, 12378604, 40686252, 168161151, 65516796, 40674602, 28240379, 66978191, 63132923, 62545344, 167915219, 28273397, 47513944, 11992086, 10265600, 17618404, 56076521, 13413281, 29505496, 22209574, 31262467, 33706145, 37969849, 63374013]",plasma membrane part
GO:0098796,-0.366,-1.959,5.537e-04,0.004,171,56,"[1994037, 32363526, 1809293, 268176, 13568664, 63078357, 6924770, 63452797, 77351967, 10689069, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 6347838, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 65627086, 63468226, 63133221, 20629788, 63199719, 12378604, 40686252, 62354250, 168161151, 65516796, 40674602, 28240379, 66978191, 63132923, 62545344, 167915219, 165851092, 28273397, 47513944, 62344870, 11992086, 10265600, 17618404, 56076521, 13413281, 29505496, 22209574, 31262467, 33706145, 37969849, 63374013]",membrane protein complex
GO:0043234,-0.366,-1.952,1.405e-03,0.003,180,56,"[1994037, 32363526, 1809293, 268176, 13568664, 63078357, 6924770, 63452797, 77351967, 10689069, 15207893, 28360306, 63332162, 17587826, 63205501, 32955361, 6347838, 31357673, 32157942, 13841186, 13203808, 62946388, 28669498, 61865720, 63129880, 65627086, 63468226, 63133221, 20629788, 63199719, 12378604, 40686252, 62354250, 168161151, 65516796, 40674602, 28240379, 66978191, 63132923, 62545344, 167915219, 165851092, 28273397, 47513944, 62344870, 11992086, 10265600, 17618404, 56076521, 13413281, 29505496, 22